# Summative Evaluation

## SSI Complete Workflow - Issuer, Holder & Verifier

Dieses Notebook demonstriert den kompletten Self-Sovereign Identity Workflow mit drei ACA-Py Agenten:
- **Issuer Agent** (Port 8021): Gibt Credentials aus
- **Holder Agent** (Port 8031): Empfängt und speichert Credentials
- **Verifier Agent** (Port 8041): Verifiziert Presentations

### Voraussetzungen
1. VON-Network läuft: `cd ../von-network && ./manage start --wait`
2. Alle drei Agenten laufen: `docker-compose up -d`
3. Python-Pakete installiert: `pip install requests pandas matplotlib plotly networkx`

## Teil 1: Setup & Vorbereitung

In [211]:
# Cell 1: Imports und Konfiguration
import requests
import json
import time
import pandas as pd
from IPython.display import display, Markdown, HTML
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# SSL Warnung für self-signed Zertifikate unterdrücken (PQC Proxy)
# import urllib3
# urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Agent URLs
ISSUER_ADMIN_URL = "https://localhost:8021"
HOLDER_ADMIN_URL = "https://localhost:8031"
VERIFIER_ADMIN_URL = "https://localhost:8041"
VON_NETWORK_URL = "https://localhost:8000"  # PQC HTTPS Reverse Proxy

# Farben für Visualisierungen
ISSUER_COLOR = "#3498db"  # Blau
HOLDER_COLOR = "#2ecc71"  # Grün
VERIFIER_COLOR = "#e74c3c"  # Rot

print("✅ Imports erfolgreich")
print(f"📍 Issuer:   {ISSUER_ADMIN_URL}")
print(f"📍 Holder:   {HOLDER_ADMIN_URL}")
print(f"📍 Verifier: {VERIFIER_ADMIN_URL}")
print(f"📍 Ledger:   {VON_NETWORK_URL}")

# ========================================
# Tails-Server Konfiguration (für Revocation)
# ========================================
TAILS_SERVER_URL = "https://localhost:6543"
TAILS_FILE_COUNT = 100  # Max Credentials per Registry

print(f"📍 Tails:    {TAILS_SERVER_URL}")
print(f"   Max Credentials per Registry: {TAILS_FILE_COUNT}")

# ========================================
# Helper Functions
# ========================================

def api_get(base_url, path):
    """GET Request zu ACA-Py Admin API"""
    url = f"{base_url}{path}"
    response = requests.get(url)
    return response.json() if response.status_code == 200 else None

def api_post(base_url, path, data):
    """POST Request zu ACA-Py Admin API"""
    url = f"{base_url}{path}"
    headers = {"Content-Type": "application/json"}
    response = requests.post(url, json=data, headers=headers)
    return response.json() if response.status_code == 200 else None

def pretty_print(data, title=""):
    """Pretty print JSON data"""
    if title:
        print(f"\n{'='*60}")
        print(f"  {title}")
        print('='*60)
    print(json.dumps(data, indent=2))
    print()

print("\n✅ Setup komplett!")


✅ Imports erfolgreich
📍 Issuer:   https://localhost:8021
📍 Holder:   https://localhost:8031
📍 Verifier: https://localhost:8041
📍 Ledger:   https://localhost:8000
📍 Tails:    https://localhost:6543
   Max Credentials per Registry: 100

✅ Setup komplett!


In [212]:
# Cell 2: Helper-Funktionen für API-Calls

def api_get(url, path):
    """GET Request an ACA-Py Admin API"""
    try:
        response = requests.get(f"{url}{path}")
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"❌ GET Fehler: {e}")
        return None

def api_post(url, path, data=None):
    """POST Request an ACA-Py Admin API"""
    try:
        response = requests.post(
            f"{url}{path}",
            json=data,
            headers={"Content-Type": "application/json"}
        )
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"❌ POST Fehler: {e}")
        if hasattr(e, 'response') and e.response:
            print(f"   Response: {e.response.text}")
        return None

def pretty_print(data, title=""):
    """Formatierte JSON-Ausgabe"""
    if title:
        print(f"\n{'='*60}")
        print(f"  {title}")
        print(f"{'='*60}")
    print(json.dumps(data, indent=2))

def wait_for_status(url, path, field, expected_value, timeout=30, interval=1):
    """Wartet bis ein Feld einen bestimmten Wert hat"""
    start_time = time.time()
    while time.time() - start_time < timeout:
        response = api_get(url, path)
        if response and response.get(field) == expected_value:
            return response
        time.sleep(interval)
    print(f"⚠️  Timeout: {field} erreichte nicht '{expected_value}' nach {timeout}s")
    return None

def status_badge(status, label="Status"):
    """HTML Badge für Status"""
    colors = {
        "active": "green",
        "completed": "green",
        "done": "green",
        "ready": "green",
        "pending": "orange",
        "failed": "red",
        "error": "red"
    }
    color = colors.get(status.lower(), "gray")
    return f'<span style="background-color:{color};color:white;padding:3px 8px;border-radius:3px;font-weight:bold">{label}: {status}</span>'

print("✅ Helper-Funktionen geladen")

✅ Helper-Funktionen geladen


In [213]:
# Cell 3: Agenten-Status prüfen (Health Check)

def check_agent_status(name, url):
    """Prüft ob ein Agent bereit ist"""
    try:
        response = api_get(url, "/status/ready")
        if response and response.get("ready"):
            return f"✅ {name}: Bereit"
        else:
            return f"⚠️  {name}: Nicht bereit"
    except:
        return f"❌ {name}: Nicht erreichbar"

print("🔍 Agenten-Status Check...\n")
print(check_agent_status("Issuer", ISSUER_ADMIN_URL))
print(check_agent_status("Holder", HOLDER_ADMIN_URL))
print(check_agent_status("Verifier", VERIFIER_ADMIN_URL))

# Detaillierte Informationen mit Conductor-Statistiken
print("\n📊 Detaillierte Agent-Informationen:\n")
agents_info = []
for name, url in [("Issuer", ISSUER_ADMIN_URL), ("Holder", HOLDER_ADMIN_URL), ("Verifier", VERIFIER_ADMIN_URL)]:
    status = api_get(url, "/status")
    if status:
        conductor = status.get("conductor", {})
        agents_info.append({
            "Agent": name,
            "Label": status.get("label", "N/A"),
            "Version": status.get("version", "N/A"),
            "In Sessions": conductor.get("in_sessions", 0),
            "Out Encode": conductor.get("out_encode", 0),
            "Out Deliver": conductor.get("out_deliver", 0),
            "Active": conductor.get("task_active", 0),
            "Done": conductor.get("task_done", 0),
            "Failed": conductor.get("task_failed", 0),
            "Pending": conductor.get("task_pending", 0)
        })

df = pd.DataFrame(agents_info)
display(df)

🔍 Agenten-Status Check...

✅ Issuer: Bereit
✅ Holder: Bereit
✅ Verifier: Bereit

📊 Detaillierte Agent-Informationen:



,Agent,Label,Version,In Sessions,Out Encode,Out Deliver,Active,Done,Failed,Pending
0,Issuer,Issuer Agent,1.3.2,0,0,0,1,2,0,0
1,Holder,Holder Agent,1.3.2,0,0,0,1,2,0,0
2,Verifier,Verifier Agent,1.3.2,0,0,0,1,2,0,0


In [214]:
# Cell 4: VON-Network Verbindung testen

print("🔗 VON-Network Verbindung testen...\n")

try:
    # Genesis-Datei abrufen
    genesis_response = requests.get(f"{VON_NETWORK_URL}/genesis")
    if genesis_response.status_code == 200:
        print("✅ VON-Network erreichbar")
        print(f"   Genesis-Datei: {len(genesis_response.text)} Bytes")
    
    # Ledger-Status abrufen
    status_response = requests.get(f"{VON_NETWORK_URL}/status")
    if status_response.status_code == 200:
        print("✅ Ledger-Status abrufbar")
        
except Exception as e:
    print(f"❌ VON-Network nicht erreichbar: {e}")
    print("   Bitte starte VON-Network: cd ../von-network && ./manage start --wait")

🔗 VON-Network Verbindung testen...

✅ VON-Network erreichbar
   Genesis-Datei: 3099 Bytes
✅ Ledger-Status abrufbar


## Teil 2: Ledger-Verbindung & DIDs

In diesem Abschnitt erstellen und registrieren wir DIDs für Issuer und Verifier auf dem Ledger.
Der Holder benötigt keine Public DID, da er nur Peer-DIDs für Connections nutzt.

In [215]:
# Cell 5: Performance Tracking Setup

# Dictionary um alle Performance-Metriken zu sammeln
performance_metrics = {
    "did_creation": [],
    "ledger_registration": [],
    "schema_creation": [],
    "cred_def_creation": [],
    "connection_establishment": [],
    "credential_issuance": [],
    "proof_request": [],
    "proof_presentation": [],
    "proof_verification": []
}

def measure_time(operation_name):
    """Decorator für Performance-Messung"""
    def decorator(func):
        def wrapper(*args, **kwargs):
            start_time = time.time()
            result = func(*args, **kwargs)
            end_time = time.time()
            duration = end_time - start_time
            performance_metrics[operation_name].append(duration)
            print(f"⏱️  {operation_name}: {duration:.3f}s")
            return result
        return wrapper
    return decorator

print("✅ Performance Tracking aktiviert")

✅ Performance Tracking aktiviert


In [216]:
# Cell 6: Issuer - DID erstellen

print("🔑 Issuer: DID erstellen...\n")

# DID-Erstellungs-Parameter für did:indy
# API: POST /did/indy/create
did_params = {
    "options": {
        "key_type": "ed25519",  # Kryptografischer Key-Type (EdDSA)
        "seed": "000000000000000000000000Issuer01"  # 32 Zeichen für reproduzierbare DID
    }
}

print(f"📋 DID-Parameter:")
print(f"   Method:   indy")
print(f"   Endpoint: /did/indy/create")
print(f"   Key Type: {did_params['options']['key_type']}")
print(f"   Seed:     {'(zufällig)' if 'seed' not in did_params['options'] else did_params['options']['seed']}")
print(f"   Format:   did:indy:<identifier>\n")

# DID erstellen mit korrektem Endpunkt
issuer_did_response = api_post(
    ISSUER_ADMIN_URL,
    "/did/indy/create",  # Korrekter Endpunkt für did:indy
    did_params
)

if issuer_did_response is not None:
    # Response-Format: {"did": "did:indy:...", "verkey": "..."}
    issuer_did = issuer_did_response.get("did")
    issuer_verkey = issuer_did_response.get("verkey")
    
    print(f"✅ Issuer DID erstellt:")
    print(f"   DID:    {issuer_did}")
    print(f"   Verkey: {issuer_verkey}")
    
    # Zeige vollständige Response
    pretty_print(issuer_did_response, "DID-Erstellungs-Response (Issuer)")
    
    # Speichern für spätere Verwendung
    issuer_info = {
        "did": issuer_did,
        "verkey": issuer_verkey,
        "method": "indy",
        "key_type": did_params["options"]["key_type"],
        "role": "ENDORSER"
    }
    
    print(f"\n💡 Format: {issuer_did} (vollständige qualified DID)")
else:
    print("❌ Fehler beim Erstellen der Issuer DID")

🔑 Issuer: DID erstellen...

📋 DID-Parameter:
   Method:   indy
   Endpoint: /did/indy/create
   Key Type: ed25519
   Seed:     000000000000000000000000Issuer01
   Format:   did:indy:<identifier>

✅ Issuer DID erstellt:
   DID:    did:indy:9pbXiFBZZGwXKp61HQBz3J
   Verkey: 2zoa6G7aMfX8GnUEpDxxunFHE7fZktRiiHk1vgMRH2tm

  DID-Erstellungs-Response (Issuer)
{
  "did": "did:indy:9pbXiFBZZGwXKp61HQBz3J",
  "verkey": "2zoa6G7aMfX8GnUEpDxxunFHE7fZktRiiHk1vgMRH2tm"
}

💡 Format: did:indy:9pbXiFBZZGwXKp61HQBz3J (vollständige qualified DID)


In [217]:
# Cell 7: Issuer DID auf Ledger registrieren

print("📝 Issuer DID auf Ledger registrieren (via VON-Network)...\n")

start_time = time.time()

# Registrierungs-Daten für VON-Network (DID-Based Registration)
# Nutzt DID+Verkey aus Cell 6 statt Seed
register_data = {
    "did": issuer_did,        # DID aus Cell 6 (ACA-Py Wallet)
    "verkey": issuer_verkey,  # Verkey aus Cell 6 (ACA-Py Wallet)
    "alias": "Issuer Agent",
    "role": "ENDORSER"  # TRUST_ANCHOR oder ENDORSER für Issuer-Rechte
}

print(f"📋 Registrierungs-Parameter:")
print(f"   Ledger URL: {VON_NETWORK_URL}/register")
print(f"   DID:        {register_data['did']}")
print(f"   Verkey:     {register_data['verkey'][:20]}...")
print(f"   Alias:      {register_data['alias']}")
print(f"   Role:       {register_data['role']}\n")

# Registriere DID auf VON-Network Ledger
# Wie in demo code: resp = await self.client_session.post(von_network_url + "/register", json=data)
print(f"🌐 POST {VON_NETWORK_URL}/register")

try:
    response = requests.post(
        f"{VON_NETWORK_URL}/register",
        json=register_data,
        timeout=10
    )
    
    response.raise_for_status()
    
    # Response format: {"did": "did:indy:...", "verkey": "..."}
    nym_info = response.json()
    
    issuer_did = nym_info["did"]
    issuer_verkey = nym_info["verkey"]
    
    duration = time.time() - start_time
    performance_metrics["ledger_registration"].append(duration)
    
    print(f"✅ Issuer DID auf Ledger registriert")
    print(f"   DID:    {issuer_did}")
    print(f"   Verkey: {issuer_verkey}")
    print(f"⏱️  Zeit: {duration:.3f}s")
    
    # Zeige vollständige Ledger-Response
    pretty_print(nym_info, "VON-Network Registration Response (Issuer)")
    
    # JETZT: Setze DID als Public DID im ACA-Py Wallet
    # Wie in demo code: await self.admin_POST("/wallet/did/public?did=" + self.did)
    print(f"\n🔑 Setze DID als Public DID im Wallet...")
    public_did_response = api_post(
        ISSUER_ADMIN_URL,
        f"/wallet/did/public?did={issuer_did}"
    )
    
    if public_did_response is not None:
        print(f"✅ Issuer DID als Public DID gesetzt")
    else:
        print(f"⚠️  Warnung: Public DID konnte nicht gesetzt werden")
    
    # Speichern für spätere Verwendung
    issuer_info = {
        "did": issuer_did,
        "verkey": issuer_verkey,
        "method": "indy",
        "key_type": "ed25519",
        "role": "ENDORSER"
    }
    
    # Hole NYM Role vom Ledger zur Verifikation
    print(f"\n🔍 NYM Role vom Ledger verifizieren...")
    nym_role_response = api_get(
        ISSUER_ADMIN_URL,
        f"/ledger/get-nym-role?did={issuer_did}"
    )
    
    if nym_role_response is not None:
        print(f"✅ NYM Role vom Ledger abgerufen")
        pretty_print(nym_role_response, "Ledger NYM Role (Issuer DID)")
        
        # Extrahiere Role
        role = nym_role_response.get('role', 'N/A')
        print(f"\n📋 Verifizierte Rolle: {role}")
    else:
        print(f"⚠️  NYM Role konnte nicht abgerufen werden")
        
except requests.exceptions.RequestException as e:
    print(f"❌ POST Fehler: {e}")
    print(f"   Stelle sicher dass VON-Network läuft: cd von-network && ./manage start")


📝 Issuer DID auf Ledger registrieren (via VON-Network)...

📋 Registrierungs-Parameter:
   Ledger URL: https://localhost:8000/register
   DID:        did:indy:9pbXiFBZZGwXKp61HQBz3J
   Verkey:     2zoa6G7aMfX8GnUEpDxx...
   Alias:      Issuer Agent
   Role:       ENDORSER

🌐 POST https://localhost:8000/register
✅ Issuer DID auf Ledger registriert
   DID:    did:indy:9pbXiFBZZGwXKp61HQBz3J
   Verkey: 2zoa6G7aMfX8GnUEpDxxunFHE7fZktRiiHk1vgMRH2tm
⏱️  Zeit: 0.086s

  VON-Network Registration Response (Issuer)
{
  "did": "did:indy:9pbXiFBZZGwXKp61HQBz3J",
  "seed": null,
  "verkey": "2zoa6G7aMfX8GnUEpDxxunFHE7fZktRiiHk1vgMRH2tm"
}

🔑 Setze DID als Public DID im Wallet...
✅ Issuer DID als Public DID gesetzt

🔍 NYM Role vom Ledger verifizieren...
✅ NYM Role vom Ledger abgerufen

  Ledger NYM Role (Issuer DID)
{
  "role": "ENDORSER"
}

📋 Verifizierte Rolle: ENDORSER


In [218]:
# Cell 8: Verifier - DID erstellen und registrieren

print("🔑 Verifier: DID erstellen und registrieren...\n")

start_time = time.time()

# DID-Erstellungs-Parameter für did:indy
# API: POST /did/indy/create
did_params = {
    "options": {
        "key_type": "ed25519"  # Kryptografischer Key-Type (EdDSA)
        # "seed": "00000000000000000000000Verifier1"  # Optional: 32 Zeichen für reproduzierbare DIDs
    }
}

print(f"📋 DID-Parameter:")
print(f"   Method:   indy")
print(f"   Endpoint: /did/indy/create")
print(f"   Key Type: {did_params['options']['key_type']}")
print(f"   Seed:     {'(zufällig)' if 'seed' not in did_params['options'] else did_params['options']['seed']}")
print(f"   Format:   did:indy:<identifier>\n")

# DID erstellen mit korrektem Endpunkt
verifier_did_response = api_post(
    VERIFIER_ADMIN_URL,
    "/did/indy/create",  # Korrekter Endpunkt für did:indy
    did_params
)

if verifier_did_response is not None:
    # Response-Format: {"did": "did:indy:...", "verkey": "..."}
    verifier_did = verifier_did_response.get("did")
    verifier_verkey = verifier_did_response.get("verkey")
    
    duration_create = time.time() - start_time
    performance_metrics["did_creation"].append(duration_create)
    
    print(f"✅ Verifier DID erstellt:")
    print(f"   DID:    {verifier_did}")
    print(f"   Verkey: {verifier_verkey}")
    print(f"⏱️  Zeit: {duration_create:.3f}s\n")
    
    # Registriere auf VON-Network Ledger
    # Wie in demo code: resp = await self.client_session.post(von_network_url + "/register", json=data)
    print("📝 Registriere NYM auf VON-Network Ledger...")
    start_time = time.time()
    
    register_data = {
        "did": verifier_did,
        "verkey": verifier_verkey,
        "alias": "Verifier Agent",
        "role": "ENDORSER"
    }
    
    print(f"📋 Registrierungs-Parameter:")
    print(f"   Ledger URL: {VON_NETWORK_URL}/register")
    print(f"   DID:        {verifier_did}")
    print(f"   Alias:      {register_data['alias']}")
    print(f"   Role:       {register_data['role']}\n")
    
    print(f"🌐 POST {VON_NETWORK_URL}/register")
    
    try:
        response = requests.post(
            f"{VON_NETWORK_URL}/register",
            json=register_data,
            timeout=10
        )
        
        response.raise_for_status()
        
        nym_info = response.json()
        
        duration_register = time.time() - start_time
        performance_metrics["ledger_registration"].append(duration_register)
        
        print(f"✅ Verifier DID auf Ledger registriert")
        print(f"⏱️  Zeit: {duration_register:.3f}s")
        
        # Zeige vollständige Ledger-Response
        pretty_print(nym_info, "VON-Network Registration Response (Verifier)")
        
        # JETZT: Setze DID als Public DID im ACA-Py Wallet
        print(f"\n🔑 Setze DID als Public DID im Wallet...")
        public_did_response = api_post(
            VERIFIER_ADMIN_URL,
            f"/wallet/did/public?did={verifier_did}"
        )
        
        if public_did_response is not None:
            print(f"✅ Verifier DID als Public DID gesetzt")
        else:
            print(f"⚠️  Warnung: Public DID konnte nicht gesetzt werden")
        
        verifier_info = {
            "did": verifier_did,
            "verkey": verifier_verkey,
            "method": "indy",
            "key_type": did_params["options"]["key_type"],
            "role": "ENDORSER"
        }
        
        # Warte kurz damit die Transaction auf dem Ledger verfügbar ist
        time.sleep(1)
        
        # Hole NYM Role vom Ledger zur Verifikation
        print(f"\n🔍 NYM Role vom Ledger verifizieren...")
        nym_role_response = api_get(
            VERIFIER_ADMIN_URL,
            f"/ledger/get-nym-role?did={verifier_did}"
        )
        
        if nym_role_response is not None:
            print(f"✅ NYM Role vom Ledger abgerufen")
            pretty_print(nym_role_response, "Ledger NYM Role (Verifier DID)")
            
            # Extrahiere Role
            role = nym_role_response.get('role', 'N/A')
            print(f"\n📋 Verifizierte Rolle: {role}")
        else:
            print(f"⚠️  NYM Role konnte nicht abgerufen werden")
        
        print(f"\n💡 Format: {verifier_did} (vollständige qualified DID)")
        
    except requests.exceptions.RequestException as e:
        print(f"❌ POST Fehler: {e}")
        print(f"   Stelle sicher dass VON-Network läuft: cd von-network && ./manage start")
else:
    print("❌ Fehler beim Erstellen der Verifier DID")

🔑 Verifier: DID erstellen und registrieren...

📋 DID-Parameter:
   Method:   indy
   Endpoint: /did/indy/create
   Key Type: ed25519
   Seed:     (zufällig)
   Format:   did:indy:<identifier>

✅ Verifier DID erstellt:
   DID:    did:indy:DuLeEuX4wg3EFiEmtz6fcU
   Verkey: 8eXpYXH2tQRvkHv5XXX8r5nCS4CabRP6i2gnyeSzEEFL
⏱️  Zeit: 0.036s

📝 Registriere NYM auf VON-Network Ledger...
📋 Registrierungs-Parameter:
   Ledger URL: https://localhost:8000/register
   DID:        did:indy:DuLeEuX4wg3EFiEmtz6fcU
   Alias:      Verifier Agent
   Role:       ENDORSER

🌐 POST https://localhost:8000/register
✅ Verifier DID auf Ledger registriert
⏱️  Zeit: 2.924s

  VON-Network Registration Response (Verifier)
{
  "did": "did:indy:DuLeEuX4wg3EFiEmtz6fcU",
  "seed": null,
  "verkey": "8eXpYXH2tQRvkHv5XXX8r5nCS4CabRP6i2gnyeSzEEFL"
}

🔑 Setze DID als Public DID im Wallet...
✅ Verifier DID als Public DID gesetzt

🔍 NYM Role vom Ledger verifizieren...
✅ NYM Role vom Ledger abgerufen

  Ledger NYM Role (Verifier 

In [219]:
# Cell 9: Domain Ledger Transaktionen

print("📋 Domain Ledger - Letzte Transaktionen")
print("🔒 Verbindung über PQC HTTPS Reverse Proxy\n")

try:
    # Hole alle Domain Ledger Transaktionen vom VON-Network
    ledger_response = requests.get(f"{VON_NETWORK_URL}/ledger/domain")
    
    if ledger_response.status_code == 200:
        # FIX: Response ist ein paginiertes Objekt, nicht direkt ein Array
        ledger_data = ledger_response.json()
        ledger_txns = ledger_data.get('results', [])
        total_txns = ledger_data.get('total', 0)
        page = ledger_data.get('page', 1)
        page_size = ledger_data.get('page_size', 100)
        
        print(f"✅ Domain Ledger abgerufen")
        print(f"   Total Transaktionen: {total_txns}")
        print(f"   Seite: {page} (Page Size: {page_size})\n")
        
        # Zeige die letzten 10 Transaktionen
        recent_txns = ledger_txns[-10:] if len(ledger_txns) > 10 else ledger_txns
        
        print("📊 Letzte 10 Transaktionen:\n")
        txn_summary = []
        
        for txn in recent_txns:
            # FIX: Korrekte verschachtelte Struktur-Zugriffe
            txn_metadata = txn.get('txnMetadata', {})
            txn_data = txn.get('txn', {})
            txn_type = txn_data.get('type', 'N/A')
            
            # Transaction Type Namen
            type_names = {
                '1': 'NYM',
                '100': 'ATTRIB',
                '101': 'SCHEMA',
                '102': 'CRED_DEF',
                '113': 'REVOC_REG_DEF',
                '114': 'REVOC_REG_ENTRY'
            }
            
            type_name = type_names.get(str(txn_type), f"Type {txn_type}")
            
            # FIX: Extrahiere 'from' aus txn.metadata (nicht txn_metadata)
            requester = txn_data.get('metadata', {}).get('from', 'N/A')
            #requester_short = requester[:20] + '...' if len(requester) > 20 else requester
            
            txn_summary.append({
                "Seq No": txn_metadata.get('seqNo', 'N/A'),
                "Type": type_name,
                "Transaction Time": txn_metadata.get('txnTime', 'N/A'),
                "Requester": requester
            })
        
        if txn_summary:
            df_txns = pd.DataFrame(txn_summary)
            display(df_txns)
        
        # Detaillierte Ansicht der letzten Transaction
        if recent_txns:
            print("\n🔍 Detaillierte Ansicht der letzten Transaction:")
            pretty_print(recent_txns[-1], "Letzte Ledger-Transaction")
    else:
        print(f"⚠️  Konnte Ledger nicht abrufen: {ledger_response.status_code}")
        
except Exception as e:
    print(f"❌ Fehler beim Abrufen des Ledgers: {e}")

print("\n💡 Transaction Types:")
print("   • NYM (1)           = DID Registration")
print("   • ATTRIB (100)      = DID Attribute")
print("   • SCHEMA (101)      = Schema Definition")
print("   • CRED_DEF (102)    = Credential Definition")
print("   • REVOC_REG_* (113/114) = Revocation Registry")

📋 Domain Ledger - Letzte Transaktionen
🔒 Verbindung über PQC HTTPS Reverse Proxy

✅ Domain Ledger abgerufen
   Total Transaktionen: 9
   Seite: 1 (Page Size: 100)

📊 Letzte 10 Transaktionen:



,Seq No,Type,Transaction Time,Requester
0,1,NYM,N/A,N/A
1,2,NYM,N/A,V4SGRU86Z58d6TV7PBUe6f
2,3,NYM,N/A,V4SGRU86Z58d6TV7PBUe6f
3,4,NYM,N/A,V4SGRU86Z58d6TV7PBUe6f
4,5,NYM,N/A,V4SGRU86Z58d6TV7PBUe6f
5,6,NYM,1765128186,V4SGRU86Z58d6TV7PBUe6f
6,7,ATTRIB,1765128189,9pbXiFBZZGwXKp61HQBz3J
7,8,NYM,1765128192,V4SGRU86Z58d6TV7PBUe6f
8,9,ATTRIB,1765128195,DuLeEuX4wg3EFiEmtz6fcU



🔍 Detaillierte Ansicht der letzten Transaction:

  Letzte Ledger-Transaction
{
  "auditPath": [
    "3eEWudnKcUWUnSTosxfTqkdbG4Ai89iPcJAQaXUXhMcv"
  ],
  "ledgerSize": 9,
  "reqSignature": {
    "type": "ED25519",
    "values": [
      {
        "from": "DuLeEuX4wg3EFiEmtz6fcU",
        "value": "2BxJJnCzpLyJAQm5Z4DMUpPqs8qxm19s5YJ8D4v8LerNJ7GpTio2fYd6AQMGGfy7ohgn82LRRvYzYAYzBXkATpe5"
      }
    ]
  },
  "rootHash": "FatvMUGrzJbHUgtvS6KQ4yKLiwGc3hGdjr6GtgJXeUPp",
  "txn": {
    "data": {
      "dest": "DuLeEuX4wg3EFiEmtz6fcU",
      "raw": "{\"endpoint\":{\"endpoint\":\"https://host.docker.internal:8040\",\"routingKeys\":[]}}"
    },
    "metadata": {
      "digest": "3f596d6f72b9506ec46c4dd3716520ce373950a0043f108f9edc8561718fdc95",
      "from": "DuLeEuX4wg3EFiEmtz6fcU",
      "payloadDigest": "1e25876ad48db3d5352bd36834710d2faa2e5576073497300de2cc6a77b7feab",
      "reqId": 1765128192402656441
    },
    "protocolVersion": 2,
    "type": "100"
  },
  "txnMetadata": {
    "seqNo": 

In [220]:
# Cell 10: Ledger-Übersicht - Alle DIDs anzeigen

print("📋 Ledger-Übersicht - Alle DIDs:\n")

# Alle DIDs von allen Agenten abrufen
issuer_dids = api_get(ISSUER_ADMIN_URL, "/wallet/did")
holder_dids = api_get(HOLDER_ADMIN_URL, "/wallet/did")
verifier_dids = api_get(VERIFIER_ADMIN_URL, "/wallet/did")

# Zeige originale Responses
print("🔍 Originale /wallet/did Responses:\n")
if issuer_dids:
    pretty_print(issuer_dids, "Issuer Wallet DIDs")
if holder_dids:
    pretty_print(holder_dids, "Holder Wallet DIDs")
if verifier_dids:
    pretty_print(verifier_dids, "Verifier Wallet DIDs")

# Erstelle erweiterte Übersicht
print("\n📊 DID-Übersicht (Detailliert):\n")
ledger_overview = []

# Issuer DIDs
if issuer_dids and "results" in issuer_dids:
    for did_info in issuer_dids["results"]:
        posture = did_info.get("posture", "unknown")
        # Zeige alle DIDs (posted und wallet_only)
        ledger_overview.append({
            "Agent": "Issuer",
            "DID": did_info["did"],
            "Verkey": did_info.get("verkey", "N/A"),
            "Posture": posture,
            "Method": did_info.get("method", "sov"),
            "Key Type": did_info.get("key_type", "N/A"),
            "On Ledger": "✅" if posture in ["public", "posted"] else "❌",
            "Endpoint": did_info.get("metadata", {}).get("endpoint", "N/A")
        })

# Holder DIDs
if holder_dids and "results" in holder_dids:
    for did_info in holder_dids["results"]:
        posture = did_info.get("posture", "unknown")
        ledger_overview.append({
            "Agent": "Holder",
            "DID": did_info["did"],
            "Verkey": did_info.get("verkey", "N/A"),
            "Posture": posture,
            "Method": did_info.get("method", "sov"),
            "Key Type": did_info.get("key_type", "N/A"),
            "On Ledger": "✅" if posture in ["public", "posted"] else "❌",
            "Endpoint": did_info.get("metadata", {}).get("endpoint", "N/A")
        })

# Verifier DIDs
if verifier_dids and "results" in verifier_dids:
    for did_info in verifier_dids["results"]:
        posture = did_info.get("posture", "unknown")
        ledger_overview.append({
            "Agent": "Verifier",
            "DID": did_info["did"],
            "Verkey": did_info.get("verkey", "N/A"),
            "Posture": posture,
            "Method": did_info.get("method", "sov"),
            "Key Type": did_info.get("key_type", "N/A"),
            "On Ledger": "✅" if posture in ["public", "posted"] else "❌",
            "Endpoint": did_info.get("metadata", {}).get("endpoint", "N/A")
        })

# Als DataFrame anzeigen
if ledger_overview:
    df_ledger = pd.DataFrame(ledger_overview)
    display(df_ledger)
else:
    print("⚠️  Keine DIDs gefunden")

# Zusammenfassung
print("\n💡 Posture-Bedeutung:")
print("   • posted       = DID auf Ledger registriert und veröffentlicht")
print("   • public       = DID als Public DID gesetzt")
print("   • wallet_only  = DID nur im Wallet, nicht auf Ledger")
print("\n💡 Hinweis:")
print("   - Issuer & Verifier benötigen Public DIDs für Schema/CredDef")
print("   - Holder nutzt nur Peer-DIDs für sichere P2P-Verbindungen")
print("   - Alte DIDs (wallet_only) können aus vergangenen Runs stammen")

📋 Ledger-Übersicht - Alle DIDs:

🔍 Originale /wallet/did Responses:


  Issuer Wallet DIDs
{
  "results": [
    {
      "did": "did:indy:9pbXiFBZZGwXKp61HQBz3J",
      "verkey": "2zoa6G7aMfX8GnUEpDxxunFHE7fZktRiiHk1vgMRH2tm",
      "posture": "posted",
      "key_type": "ed25519",
      "method": "indy",
      "metadata": {
        "posted": true,
        "endpoint": "https://host.docker.internal:8020"
      }
    }
  ]
}

  Holder Wallet DIDs
{
  "results": []
}

  Verifier Wallet DIDs
{
  "results": [
    {
      "did": "did:indy:DuLeEuX4wg3EFiEmtz6fcU",
      "verkey": "8eXpYXH2tQRvkHv5XXX8r5nCS4CabRP6i2gnyeSzEEFL",
      "posture": "posted",
      "key_type": "ed25519",
      "method": "indy",
      "metadata": {
        "posted": true,
        "endpoint": "https://host.docker.internal:8040"
      }
    }
  ]
}

📊 DID-Übersicht (Detailliert):



,Agent,DID,Verkey,Posture,Method,Key Type,On Ledger,Endpoint
0,Issuer,did:indy:9pbXiFBZZGwXKp61HQBz3J,2zoa6G7aMfX8GnUEpDxxunFHE7fZktRiiHk1vgMRH2tm,posted,indy,ed25519,✅,https://host.docker.internal:8020
1,Verifier,did:indy:DuLeEuX4wg3EFiEmtz6fcU,8eXpYXH2tQRvkHv5XXX8r5nCS4CabRP6i2gnyeSzEEFL,posted,indy,ed25519,✅,https://host.docker.internal:8040



💡 Posture-Bedeutung:
   • posted       = DID auf Ledger registriert und veröffentlicht
   • public       = DID als Public DID gesetzt
   • wallet_only  = DID nur im Wallet, nicht auf Ledger

💡 Hinweis:
   - Issuer & Verifier benötigen Public DIDs für Schema/CredDef
   - Holder nutzt nur Peer-DIDs für sichere P2P-Verbindungen
   - Alte DIDs (wallet_only) können aus vergangenen Runs stammen


## Teil 3: Schema & Credential Definition - KRITIS-Notfall-Wartungszertifikat

In diesem Abschnitt wird das **Schema** für KRITIS-Notfall-Wartungszertifikate auf der Hyperledger Indy Blockchain registriert. Das Schema definiert die Struktur des Credentials mit 9 Attributen:

**Identitätsattribute**:
- `first_name`, `name`: Vor- und Nachname des Technikers
- `organisation`: Arbeitgeber/Dienstleister
- `role`: Berufliche Funktion (z.B. "Notfalltechniker")

**Berechtigungsattribute**:
- `cert_type`: Art der Berechtigung (z.B. "Notfall-Wartungsberechtigung")
- `facility_type`: Autorisierte Anlage (z.B. "Umspannwerk Nord-Ost")
- `security_access_level`: Sicherheitsfreigabe (z.B. "Stufe-3-Kritisch")

**Zeitliche Attribute**:
- `epoch_valid_from`: Gültigkeitsbeginn (Unix Epoch Timestamp)
- `epoch_valid_until`: Gültigkeitsende (Unix Epoch Timestamp)

**KRITIS-Konformität**: Das Schema folgt den Anforderungen der EU-Verordnung 2019/881 (Cybersecurity Act) für kritische Infrastruktur im Energiesektor.

In [ ]:
# Cell 11: Schema erstellen (Indy)

print("📋 Schema erstellen...\n")

start_time = time.time()

# Schema-Daten für KRITIS-Notfall-Wartungszertifikat (Indy API)
# API: POST /schemas
# Format: {"schema_name": "...", "schema_version": "...", "attributes": [...]}
schema_data = {
    "schema_name": "kritis_emergency_maintenance_cert",
    "schema_version": "1.0",
    "attributes": [
        "first_name",
        "name",
        "organisation",
        "role",
        "cert_type",
        "facility_type",
        "epoch_valid_from",
        "epoch_valid_until",
        "security_access_level"
    ]
}

print(f"📋 Schema-Parameter (KRITIS-Notfall-Wartungszertifikat):")
print(f"   Endpoint:  /schemas")
print(f"   Name:      {schema_data['schema_name']}")
print(f"   Version:   {schema_data['schema_version']}")
print(f"   Attribute: {', '.join(schema_data['attributes'])}\n")

# Schema auf Ledger registrieren (Indy endpoint)
schema_response = api_post(
    ISSUER_ADMIN_URL,
    "/schemas",  # Indy endpoint
    schema_data
)

if schema_response is not None:
    # Response Format: {"schema_id": "...", "schema": {...}}
    schema_id = schema_response.get("schema_id")
    
    duration = time.time() - start_time
    performance_metrics["schema_creation"].append(duration)
    
    print(f"✅ Schema 'kritis_emergency_maintenance_cert' erstellt (Indy)")
    print(f"   Schema ID: {schema_id}")
    print(f"⏱️  Zeit: {duration:.3f}s")
    
    # Zeige vollständige Schema-Response
    pretty_print(schema_response, "Indy Schema Response (KRITIS-Notfall-Wartungszertifikat)")
    
else:
    print("❌ Fehler beim Erstellen des Schemas")

📋 Schema erstellen...

📋 Schema-Parameter (KRITIS-Notfall-Wartungszertifikat):
   Endpoint:  /schemas
   Name:      kritis_emergency_maintenance_cert
   Version:   1.0
   Attribute: first_name, name, organisation, role, cert_type, facility_type, epoch_valid_from, epoch_valid_until, security_access_level

✅ Schema 'kritis_emergency_maintenance_cert' erstellt (Indy)
   Schema ID: 9pbXiFBZZGwXKp61HQBz3J:2:kritis_emergency_maintenance_cert:1.0
⏱️  Zeit: 0.124s

  Indy Schema Response (KRITIS-Notfall-Wartungszertifikat)
{
  "sent": {
    "schema_id": "9pbXiFBZZGwXKp61HQBz3J:2:kritis_emergency_maintenance_cert:1.0",
    "schema": {
      "ver": "1.0",
      "id": "9pbXiFBZZGwXKp61HQBz3J:2:kritis_emergency_maintenance_cert:1.0",
      "name": "kritis_emergency_maintenance_cert",
      "version": "1.0",
      "attrNames": [
        "role",
        "cert_type",
        "epoch_valid_until",
        "first_name",
        "name",
        "organisation",
        "facility_type",
        "security_a

In [222]:
# Cell 12: Credential Definition erstellen (Indy)

print("📝 Credential Definition erstellen...\n")

start_time = time.time()

# Credential Definition Data (Indy API)
# API: POST /credential-definitions
# Format: {"schema_id": "...", "tag": "...", "support_revocation": bool, ...}
cred_def_data = {
    "schema_id": schema_id,  # from Cell 11
    "tag": "default",
    "support_revocation": True,  # Enable revocation
    "revocation_registry_size": TAILS_FILE_COUNT  # from config
}

print(f"📝 Credential Definition Parameter (Indy):")
print(f"   Endpoint:            /credential-definitions")
print(f"   Schema ID:           {cred_def_data['schema_id']}")
print(f"   Tag:                 {cred_def_data['tag']}")
print(f"   Support Revocation:  {cred_def_data['support_revocation']}")
print(f"   RevReg Size:         {cred_def_data['revocation_registry_size']}\n")

# Create Credential Definition on Ledger (Indy endpoint)
cred_def_response = api_post(
    ISSUER_ADMIN_URL,
    "/credential-definitions",  # Indy endpoint
    cred_def_data
)

if cred_def_response is not None:
    # Response Format: {"credential_definition_id": "..."}
    cred_def_id = cred_def_response.get("credential_definition_id")

    duration = time.time() - start_time
    performance_metrics["cred_def_creation"].append(duration)

    print(f"✅ Credential Definition erstellt (Indy)")
    print(f"   Cred Def ID: {cred_def_id}")
    print(f"   für Schema 'kritis_emergency_maintenance_cert'")
    print(f"⏱️  Zeit: {duration:.3f}s")

    # Show full Cred Def Response
    pretty_print(cred_def_response, "Indy Credential Definition Response (KRITIS)")

else:
    print("❌ Fehler beim Erstellen der Credential Definition")

📝 Credential Definition erstellen...

📝 Credential Definition Parameter (Indy):
   Endpoint:            /credential-definitions
   Schema ID:           9pbXiFBZZGwXKp61HQBz3J:2:kritis_emergency_maintenance_cert:1.0
   Tag:                 default
   Support Revocation:  True
   RevReg Size:         100

✅ Credential Definition erstellt (Indy)
   Cred Def ID: 9pbXiFBZZGwXKp61HQBz3J:3:CL:10:default
   für Schema 'kritis_emergency_maintenance_cert'
⏱️  Zeit: 15.048s

  Indy Credential Definition Response (KRITIS)
{
  "sent": {
    "credential_definition_id": "9pbXiFBZZGwXKp61HQBz3J:3:CL:10:default"
  },
  "credential_definition_id": "9pbXiFBZZGwXKp61HQBz3J:3:CL:10:default"
}


## Teil 4: Agent Connections - KRITIS-Akteure

**Verbindungsaufbau zwischen den Akteuren**:
1. **Issuer ↔ Holder**: Energienetzbetreiber ↔ Notfalltechniker
2. **Holder ↔ Verifier**: Notfalltechniker ↔ Zutrittssystem (Umspannwerk)

**PQC-Integration**: Alle Verbindungen verwenden did:peer:4 mit Post-Quantum-Kryptographie (ML-DSA-65 für Signaturen, ML-KEM-768 für Key Agreement).

In [223]:
# Cell 13: Issuer Discovery - Identifikation via Ledger Identifier

print("🔍 Issuer Discovery - Identifikation vertrauenswürdiger Aussteller")
print("="*80)
print("📖 KRITIS-Anforderung: Identity Holder müssen vertrauenswürdige Issuers")
print("   für gewünschte Credentials entdecken und deren Identität validieren.\n")

# Phase 1: Ledger-basierte Issuer Discovery
print("📋 Phase 1: Ledger-basierte Issuer Discovery\n")

try:
    # Hole Domain Ledger Transaktionen
    ledger_response = requests.get(f"{VON_NETWORK_URL}/ledger/domain")
    
    if ledger_response.status_code == 200:
        ledger_data = ledger_response.json()
        ledger_txns = ledger_data.get('results', [])
        
        # Filter: Nur NYM-Transaktionen (DID-Registrierungen, Type '1')
        nym_txns = [
            txn for txn in ledger_txns 
            if txn.get('txn', {}).get('type') == '1'
        ]
        
        # Filter: Nur Transaktionen mit TRUST_ANCHOR Role (= Issuers)
        issuer_identities = []
        for txn in nym_txns:
            txn_data = txn.get('txn', {}).get('data', {})
            role = txn_data.get('role')
            
            # Role '101' = TRUST_ANCHOR (berechtigt, Credentials auszustellen)
            if role == '101':
                full_verkey = txn_data.get('verkey', 'N/A')
                issuer_identities.append({
                    'did': txn_data.get('dest'),
                    'verkey_full': full_verkey,
                    'verkey_short': full_verkey[:30] + '...' if len(full_verkey) > 30 else full_verkey,
                    'alias': txn_data.get('alias', 'N/A'),
                    'registered_by': txn.get('txn', {}).get('metadata', {}).get('from', 'N/A'),
                    'txn_time': txn.get('txnMetadata', {}).get('txnTime'),
                    'seq_no': txn.get('txnMetadata', {}).get('seqNo')
                })
        
        print(f"✅ {len(issuer_identities)} TRUST_ANCHOR Issuer(s) auf dem Ledger gefunden:\n")
        
        if issuer_identities:
            # Zeige nur relevante Spalten für Identifikation
            df_issuers = pd.DataFrame([{
                'Alias': i['alias'],
                'DID': i['did'],
                'Verkey (gekürzt)': i['verkey_short'],
                'Seq No': i['seq_no']
            } for i in issuer_identities])
            display(df_issuers)
        else:
            print("⚠️  Keine TRUST_ANCHOR gefunden (Ledger noch leer?)")
            
except Exception as e:
    print(f"❌ Fehler bei Issuer Discovery: {e}")
    import traceback
    traceback.print_exc()

# Phase 2: Issuer Identifier Verification
print(f"\n{'='*80}")
print("🔐 Phase 2: Issuer Identifier Verification\n")

def verify_issuer_identity(issuer_did, issuer_data, all_txns):
    """
    Verifiziert die Identität eines Issuers anhand von Ledger-Daten.
    
    Identitäts-Eigenschaften:
    1. DID (Decentralized Identifier)
    2. Verkey (Public Key für Signaturverifikation)
    3. TRUST_ANCHOR Role
    4. Registrierungs-Endorser (Wer hat die Identität bestätigt?)
    5. On-Ledger Aktivitäten (Schemas, CRED_DEFs, etc.)
    """
    identity_info = {
        'verified': True,
        'properties': []
    }
    
    # 1. DID Identifier
    identity_info['properties'].append({
        'property': 'DID (Identifier)',
        'value': issuer_did,
        'verified': True
    })
    
    # 2. Verkey (Public Key)
    verkey = issuer_data.get('verkey_full', 'N/A')
    identity_info['properties'].append({
        'property': 'Verkey (Public Key)',
        'value': verkey,
        'verified': verkey != 'N/A'
    })
    
    # 3. TRUST_ANCHOR Role
    identity_info['properties'].append({
        'property': 'TRUST_ANCHOR Role',
        'value': 'Ja (Role 101)',
        'verified': True
    })
    
    # 4. Endorser (Wer hat registriert?)
    registered_by = issuer_data.get('registered_by', 'N/A')
    identity_info['properties'].append({
        'property': 'Registriert von',
        'value': registered_by,
        'verified': registered_by != 'N/A'
    })
    
    # 5. On-Ledger Aktivitäten
    activities = []
    
    # Schemas (Type '101' im Ledger)
    schemas = [
        txn for txn in all_txns 
        if txn.get('txn', {}).get('type') == '101'
        and txn.get('txn', {}).get('metadata', {}).get('from') == issuer_did
    ]
    if schemas:
        activities.append(f"{len(schemas)} Schema(s)")
    
    # Credential Definitions (Type '102')
    cred_defs = [
        txn for txn in all_txns 
        if txn.get('txn', {}).get('type') == '102'
        and txn.get('txn', {}).get('metadata', {}).get('from') == issuer_did
    ]
    if cred_defs:
        activities.append(f"{len(cred_defs)} CRED_DEF(s)")
    
    # Revocation Registries (Type '113' oder '114')
    revoc_regs = [
        txn for txn in all_txns 
        if txn.get('txn', {}).get('type') in ['113', '114']
        and txn.get('txn', {}).get('metadata', {}).get('from') == issuer_did
    ]
    if revoc_regs:
        activities.append(f"{len(revoc_regs)} REVOC_REG(s)")
    
    activity_summary = ', '.join(activities) if activities else 'Keine'
    identity_info['properties'].append({
        'property': 'On-Ledger Aktivitäten',
        'value': activity_summary,
        'verified': len(activities) > 0
    })
    
    # 6. Zeitstempel (Registrierungszeitpunkt)
    import datetime
    txn_time = issuer_data.get('txn_time')
    if txn_time:
        registration_date = datetime.datetime.fromtimestamp(txn_time)
        age_days = (datetime.datetime.now() - registration_date).days
        identity_info['properties'].append({
            'property': 'Registriert seit',
            'value': f"{registration_date.strftime('%Y-%m-%d')} ({age_days} Tage)",
            'verified': True
        })
    
    return identity_info

# Zeige Identifier-Details für alle gefundenen Issuers
if 'issuer_identities' in locals() and issuer_identities:
    for issuer in issuer_identities[:3]:  # Zeige nur erste 3 Issuers
        print(f"{'='*80}")
        print(f"🏢 Issuer: {issuer['alias']}")
        print(f"{'='*80}")
        
        identity = verify_issuer_identity(issuer['did'], issuer, ledger_txns)
        
        for prop in identity['properties']:
            status = "✅" if prop['verified'] else "⚠️ "
            value = prop['value']
            # Kürze lange Werte (z.B. Verkey)
            if len(str(value)) > 50:
                value = str(value)[:47] + "..."
            print(f"   {status} {prop['property']:25s} {value}")
        
        print()

# Phase 3: Issuer-Suche für spezifisches Schema
print(f"{'='*80}")
print("🔎 Phase 3: Issuer-Identifikation für Schema\n")

# Suche nach dem Schema aus Cell 11
desired_schema_name = "kritis_emergency_maintenance_cert"

try:
    # Hole alle Schemas vom Ledger (Type '101')
    schemas = [
        txn for txn in ledger_txns 
        if txn.get('txn', {}).get('type') == '101'
    ]
    
    # Filter nach Schema-Namen
    matching_schemas = []
    for schema_txn in schemas:
        schema_data = schema_txn.get('txn', {}).get('data', {}).get('data', {})
        schema_name = schema_data.get('name', '')
        
        if desired_schema_name.lower() in schema_name.lower():
            schema_version = schema_data.get('version', 'N/A')
            created_by = schema_txn.get('txn', {}).get('metadata', {}).get('from', 'N/A')
            
            matching_schemas.append({
                'schema_name': schema_name,
                'schema_version': schema_version,
                'schema_id': f"{created_by}:2:{schema_name}:{schema_version}",
                'created_by_did': created_by
            })
    
    if matching_schemas:
        print(f"✅ {len(matching_schemas)} Schema(s) gefunden für '{desired_schema_name}':\n")
        df_schemas = pd.DataFrame(matching_schemas)
        display(df_schemas)
        
        # Identifiziere die Issuers dieser Schemas
        print(f"\n{'='*80}")
        print("🔐 Issuer-Identifikation für gefundene Schemas:\n")
        
        for schema in matching_schemas:
            schema_issuer_did = schema['created_by_did']
            
            # Finde Issuer-Identität
            issuer_info = next(
                (i for i in issuer_identities if i['did'] == schema_issuer_did), 
                None
            )
            
            print(f"Schema: {schema['schema_name']} v{schema['schema_version']}")
            print(f"{'-'*80}")
            
            if issuer_info:
                # Zeige Issuer-Identifikation
                print(f"✅ Issuer identifiziert:")
                print(f"   Alias:         {issuer_info['alias']}")
                print(f"   DID:           {issuer_info['did']}")
                print(f"   Verkey:        {issuer_info['verkey_short']}")
                print(f"   TRUST_ANCHOR:  Ja (Role 101)")
                print(f"   Ledger Seq No: {issuer_info['seq_no']}")
                
                # Zeige vollständige Identitäts-Verifikation
                print(f"\n   📋 Vollständige Identitäts-Verifikation:")
                identity = verify_issuer_identity(schema_issuer_did, issuer_info, ledger_txns)
                for prop in identity['properties']:
                    status = "✅" if prop['verified'] else "⚠️ "
                    value = prop['value']
                    if len(str(value)) > 40:
                        value = str(value)[:37] + "..."
                    print(f"   {status} {prop['property']:20s} {value}")
                
                print(f"\n   ✅ Issuer erfolgreich über DID identifiziert und verifiziert")
            else:
                print(f"⚠️  Issuer NICHT als TRUST_ANCHOR registriert")
                print(f"   DID: {schema_issuer_did}")
                print(f"   Status: Keine TRUST_ANCHOR Role auf dem Ledger")
            
            print(f"\n{'='*80}\n")
    else:
        print(f"⚠️  Keine Schemas gefunden für '{desired_schema_name}'")
        print(f"💡 Tipp: Stelle sicher, dass Cell 11 (Schema erstellen) ausgeführt wurde")
        
except Exception as e:
    print(f"❌ Fehler bei Schema-Suche: {e}")
    import traceback
    traceback.print_exc()

print(f"\n{'='*80}")
print("💡 Issuer Identifier-basierte Discovery:")
print("   1. ✅ DID (Decentralized Identifier) - Eindeutiger Identifier")
print("   2. ✅ Verkey (Public Key) - Kryptografische Identität")
print("   3. ✅ TRUST_ANCHOR Role - Berechtigung zum Ausstellen")
print("   4. ✅ Ledger Endorsement - Wer hat die Identität bestätigt?")
print("   5. ✅ On-Ledger Aktivitäten - Schemas, CRED_DEFs, REVOC_REGs")
print(f"{'='*80}")
print("🎯 Identifikations-Methode:")
print("   Der Issuer wird über seinen DID (Decentralized Identifier) eindeutig")
print("   identifiziert. Der DID ist im Schema-Identifier eingebettet:")
print("   Format: <issuer_did>:2:<schema_name>:<version>")
print("   ")
print("   Die Vertrauenswürdigkeit wird durch die TRUST_ANCHOR Role auf dem")
print("   Hyperledger Indy Ledger garantiert (Byzantine Fault Tolerant).")
print(f"{'='*80}")


🔍 Issuer Discovery - Identifikation vertrauenswürdiger Aussteller
📖 KRITIS-Anforderung: Identity Holder müssen vertrauenswürdige Issuers
   für gewünschte Credentials entdecken und deren Identität validieren.

📋 Phase 1: Ledger-basierte Issuer Discovery

✅ 2 TRUST_ANCHOR Issuer(s) auf dem Ledger gefunden:



,Alias,DID,Verkey (gekürzt),Seq No
0,Issuer Agent,9pbXiFBZZGwXKp61HQBz3J,2zoa6G7aMfX8GnUEpDxxunFHE7fZkt...,6
1,Verifier Agent,DuLeEuX4wg3EFiEmtz6fcU,8eXpYXH2tQRvkHv5XXX8r5nCS4CabR...,8



🔐 Phase 2: Issuer Identifier Verification

🏢 Issuer: Issuer Agent
   ✅ DID (Identifier)          9pbXiFBZZGwXKp61HQBz3J
   ✅ Verkey (Public Key)       2zoa6G7aMfX8GnUEpDxxunFHE7fZktRiiHk1vgMRH2tm
   ✅ TRUST_ANCHOR Role         Ja (Role 101)
   ✅ Registriert von           V4SGRU86Z58d6TV7PBUe6f
   ✅ On-Ledger Aktivitäten     1 Schema(s), 1 CRED_DEF(s), 4 REVOC_REG(s)
   ✅ Registriert seit          2025-12-07 (0 Tage)

🏢 Issuer: Verifier Agent
   ✅ DID (Identifier)          DuLeEuX4wg3EFiEmtz6fcU
   ✅ Verkey (Public Key)       8eXpYXH2tQRvkHv5XXX8r5nCS4CabRP6i2gnyeSzEEFL
   ✅ TRUST_ANCHOR Role         Ja (Role 101)
   ✅ Registriert von           V4SGRU86Z58d6TV7PBUe6f
   ⚠️  On-Ledger Aktivitäten     Keine
   ✅ Registriert seit          2025-12-07 (0 Tage)

🔎 Phase 3: Issuer-Identifikation für Schema

✅ 1 Schema(s) gefunden für 'kritis_emergency_maintenance_cert':



,schema_name,schema_version,schema_id,created_by_did
0,kritis_emergency_maintenance_cert,1.0,9pbXiFBZZGwXKp61HQBz3J:2:kritis_emergency_main...,9pbXiFBZZGwXKp61HQBz3J



🔐 Issuer-Identifikation für gefundene Schemas:

Schema: kritis_emergency_maintenance_cert v1.0
--------------------------------------------------------------------------------
✅ Issuer identifiziert:
   Alias:         Issuer Agent
   DID:           9pbXiFBZZGwXKp61HQBz3J
   Verkey:        2zoa6G7aMfX8GnUEpDxxunFHE7fZkt...
   TRUST_ANCHOR:  Ja (Role 101)
   Ledger Seq No: 6

   📋 Vollständige Identitäts-Verifikation:
   ✅ DID (Identifier)     9pbXiFBZZGwXKp61HQBz3J
   ✅ Verkey (Public Key)  2zoa6G7aMfX8GnUEpDxxunFHE7fZktRiiHk1v...
   ✅ TRUST_ANCHOR Role    Ja (Role 101)
   ✅ Registriert von      V4SGRU86Z58d6TV7PBUe6f
   ✅ On-Ledger Aktivitäten 1 Schema(s), 1 CRED_DEF(s), 4 REVOC_R...
   ✅ Registriert seit     2025-12-07 (0 Tage)

   ✅ Issuer erfolgreich über DID identifiziert und verifiziert



💡 Issuer Identifier-basierte Discovery:
   1. ✅ DID (Decentralized Identifier) - Eindeutiger Identifier
   2. ✅ Verkey (Public Key) - Kryptografische Identität
   3. ✅ TRUST_ANCHOR Role - Berec

In [224]:
# Cell 14: Connection Issuer → Holder erstellen

print("🤝 Connection: Issuer → Holder erstellen...\n")

start_time = time.time()

# ========================================
# 1. PRE-CHECK: Existierende Connections prüfen
# ========================================
print("🔍 Pre-Check: Prüfe existierende Connections...\n")

existing_conn_issuer_holder = None
existing_conn_holder = None

# Prüfe Issuer-Seite: GET /connections
issuer_conns_response = api_get(ISSUER_ADMIN_URL, "/connections")
if issuer_conns_response and "results" in issuer_conns_response:
    print(f"   Issuer: {len(issuer_conns_response['results'])} Connection(s) gefunden")
    for conn in issuer_conns_response["results"]:
        if conn.get("state") == "completed":
            existing_conn_issuer_holder = conn["connection_id"]
            print(f"     → Connection {existing_conn_issuer_holder} (State: completed)")
            break
else:
    print("   Issuer: Keine Connections gefunden")

# Prüfe Holder-Seite: GET /connections
holder_conns_response = api_get(HOLDER_ADMIN_URL, "/connections")
if holder_conns_response and "results" in holder_conns_response:
    print(f"   Holder: {len(holder_conns_response['results'])} Connection(s) gefunden")
    for conn in holder_conns_response["results"]:
        if conn.get("state") == "completed":
            existing_conn_holder = conn["connection_id"]
            print(f"     → Connection {existing_conn_holder} (State: completed)")
            break
else:
    print("   Holder: Keine Connections gefunden")

if existing_conn_issuer_holder and existing_conn_holder:
    print(f"\n✅ Existierende Connection gefunden!")
    print(f"   Issuer Connection ID: {existing_conn_issuer_holder}")
    print(f"   Holder Connection ID: {existing_conn_holder}")
    print("   Überspringe Connection-Erstellung\n")
    conn_id_issuer_holder = existing_conn_issuer_holder
    conn_id_holder = existing_conn_holder
else:
    print("\n   Keine vollständige existierende Connection gefunden")
    print("   Erstelle neue Connection mit did:peer:2...\n")
    
    # ========================================
    # 2. HAUPT-WORKFLOW: Connection mit did:peer erstellen
    # ========================================
    
    # Initialisiere Variablen
    conn_id_issuer_holder = None
    conn_id_holder = None
    invitation_msg_id = None
    invitation_key = None
    
    # Issuer erstellt Out-of-Band Invitation mit did:peer:2
    invitation_data = {
        "handshake_protocols": ["https://didcomm.org/didexchange/1.1"],
        "use_did_method": "did:peer:4",
        "my_label": "Issuer Agent",
        "goal": "Establish connection for credential issuance",
        "goal_code": "issue-vc",
        "accept": [
            "didcomm/aip1",
            "didcomm/aip2;env=rfc19"
        ]
    }
    
    print("📋 Invitation-Modus: did:peer:2 (P2P Connection)")
    
    # API Call mit Query-Parametern für auto-accept
    invitation_response = api_post(
        ISSUER_ADMIN_URL,
        "/out-of-band/create-invitation?auto_accept=true&multi_use=false",
        invitation_data
    )
    
    if invitation_response and "invitation" in invitation_response:
        invitation = invitation_response["invitation"]
        oob_id = invitation_response.get("oob_id")
        invitation_url = invitation_response.get("invitation_url", "")
        state = invitation_response.get("state")
        
        # WICHTIG: Speichere invitation_msg_id aus der Invitation
        invitation_msg_id = invitation.get("@id")
        
        # Optional: Extrahiere invitation_key aus services (falls vorhanden)
        services = invitation.get("services", [])
        if services and len(services) > 0:
            # Bei did:peer:2 ist der Service oft inline
            service = services[0]
            if isinstance(service, dict):
                recipient_keys = service.get("recipientKeys", [])
                if recipient_keys:
                    invitation_key = recipient_keys[0]
        
        print(f"✅ Issuer: Invitation erstellt")
        print(f"   OOB ID: {oob_id}")
        print(f"   State: {state}")
        print(f"   Invitation Msg ID: {invitation_msg_id}")
        if invitation_key:
            print(f"   Invitation Key: {invitation_key[:30]}...")
        print(f"   Services: {len(invitation.get('services', []))} service(s)")
        print(f"   Label: {invitation_data['my_label']}")
        print(f"   Goal: {invitation_data['goal']}")
        
        # Holder akzeptiert Invitation (mit auto-accept)
        receive_response = api_post(
            HOLDER_ADMIN_URL,
            "/out-of-band/receive-invitation?auto_accept=true",
            invitation
        )
        
        if receive_response is not None:
            conn_id_holder = receive_response.get("connection_id")
            print(f"\n✅ Holder: Invitation akzeptiert")
            print(f"   Connection ID (Holder): {conn_id_holder}")
            
            # Issuer findet seine Connection via invitation_msg_id
            print(f"\n   Issuer sucht Connection via invitation_msg_id...")
            time.sleep(2)  # Kurze Wartezeit für DIDExchange Protocol
            
            # Hole alle Issuer Connections
            issuer_conns = api_get(ISSUER_ADMIN_URL, "/connections")
            if issuer_conns and "results" in issuer_conns:
                connections = issuer_conns["results"]
                
                print(f"     Prüfe {len(connections)} Connection(s)...")
                
                # Finde Connection anhand invitation_msg_id (exaktes Match!)
                connection_found = False
                for conn in connections:
                    conn_invitation_msg_id = conn.get("invitation_msg_id")
                    conn_invitation_key = conn.get("invitation_key")
                    
                    # Match via invitation_msg_id (bevorzugt)
                    if invitation_msg_id and conn_invitation_msg_id == invitation_msg_id:
                        conn_id_issuer_holder = conn["connection_id"]
                        conn_state = conn.get("state")
                        print(f"\n   ✅ Issuer: Connection gefunden via invitation_msg_id!")
                        print(f"      • Connection ID: {conn_id_issuer_holder}")
                        print(f"      • State: {conn_state}")
                        print(f"      • Invitation Msg ID: {conn_invitation_msg_id}")
                        print(f"      • Their Label: {conn.get('their_label', 'N/A')}")
                        print(f"      • Updated At: {conn.get('updated_at', 'N/A')}")
                        connection_found = True
                        break
                    
                    # Fallback: Match via invitation_key
                    elif invitation_key and conn_invitation_key == invitation_key:
                        conn_id_issuer_holder = conn["connection_id"]
                        conn_state = conn.get("state")
                        print(f"\n   ✅ Issuer: Connection gefunden via invitation_key!")
                        print(f"      • Connection ID: {conn_id_issuer_holder}")
                        print(f"      • State: {conn_state}")
                        print(f"      • Invitation Key: {conn_invitation_key[:30]}...")
                        print(f"      • Their Label: {conn.get('their_label', 'N/A')}")
                        connection_found = True
                        break
                
                if not connection_found:
                    print(f"     ⚠️  Keine Connection mit matching invitation_msg_id gefunden")
                    print(f"     Gesuchte invitation_msg_id: {invitation_msg_id}")
            else:
                print(f"     ❌ Konnte Issuer Connections nicht abrufen")
        else:
            print("❌ Holder konnte Invitation nicht akzeptieren")
    else:
        print("❌ Fehler beim Erstellen der Invitation")

# ========================================
# 3. POST-VALIDATION: Connection-Status verifizieren
# ========================================
print("\n🔍 Post-Validation: Verifiziere Connection-Status...\n")

validation_success = False
issuer_state = None
holder_state = None

if conn_id_issuer_holder:
    # Prüfe Issuer-Seite mit spezifischem Endpoint: GET /connections/{conn_id}
    issuer_conn_detail = api_get(
        ISSUER_ADMIN_URL,
        f"/connections/{conn_id_issuer_holder}"
    )
    
    if issuer_conn_detail:
        issuer_state = issuer_conn_detail.get("state")
        issuer_their_label = issuer_conn_detail.get("their_label")
        issuer_their_role = issuer_conn_detail.get("their_role")
        issuer_invitation_msg_id = issuer_conn_detail.get("invitation_msg_id")
        
        print(f"   📋 Issuer Connection {conn_id_issuer_holder}:")
        print(f"      • State: {issuer_state}")
        print(f"      • Their Label: {issuer_their_label}")
        print(f"      • Their Role: {issuer_their_role}")
        print(f"      • Invitation Msg ID: {issuer_invitation_msg_id}")

# Prüfe Holder-Seite (falls conn_id_holder bekannt)
if conn_id_holder:
    # Prüfe Holder-Seite mit spezifischem Endpoint: GET /connections/{conn_id}
    holder_conn_detail = api_get(
        HOLDER_ADMIN_URL,
        f"/connections/{conn_id_holder}"
    )
    
    if holder_conn_detail:
        holder_state = holder_conn_detail.get("state")
        holder_their_label = holder_conn_detail.get("their_label")
        holder_their_role = holder_conn_detail.get("their_role")
        holder_invitation_msg_id = holder_conn_detail.get("invitation_msg_id")
        
        print(f"\n   📋 Holder Connection {conn_id_holder}:")
        print(f"      • State: {holder_state}")
        print(f"      • Their Label: {holder_their_label}")
        print(f"      • Their Role: {holder_their_role}")
        print(f"      • Invitation Msg ID: {holder_invitation_msg_id}")
        
        # Verifiziere dass invitation_msg_id übereinstimmt
        if invitation_msg_id and issuer_invitation_msg_id == holder_invitation_msg_id:
            print(f"      ✅ Invitation Msg IDs stimmen überein!")

# Validierung
if issuer_state == "active" and holder_state == "active":
    validation_success = True
elif issuer_state == "active" and not conn_id_holder:
    # Bei existierender Connection nur Issuer-Seite bekannt
    validation_success = True

duration = time.time() - start_time
performance_metrics["connection_establishment"].append(duration)

# Finale Ausgabe
print(f"\n{'='*60}")
if validation_success:
    print(f"✅ CONNECTION ESTABLISHED (did:peer:2): Issuer ↔ Holder")
    print(f"   Issuer Connection ID: {conn_id_issuer_holder}")
    if conn_id_holder:
        print(f"   Holder Connection ID: {conn_id_holder}")
    print(f"   Issuer State: {issuer_state}")
    if holder_state:
        print(f"   Holder State: {holder_state}")
    if invitation_msg_id:
        print(f"   Invitation Msg ID: {invitation_msg_id}")
else:
    print(f"⚠️  CONNECTION STATUS UNCLEAR")
    print(f"   Issuer State: {issuer_state if issuer_state else 'N/A'}")
    print(f"   Holder State: {holder_state if holder_state else 'N/A'}")
    if conn_id_issuer_holder:
        print(f"   Issuer Connection ID: {conn_id_issuer_holder}")
    if conn_id_holder:
        print(f"   Holder Connection ID: {conn_id_holder}")

print(f"⏱️  Gesamtzeit: {duration:.3f}s")
print(f"{'='*60}")


🤝 Connection: Issuer → Holder erstellen...

🔍 Pre-Check: Prüfe existierende Connections...

   Issuer: 0 Connection(s) gefunden
   Holder: 0 Connection(s) gefunden

   Keine vollständige existierende Connection gefunden
   Erstelle neue Connection mit did:peer:2...

📋 Invitation-Modus: did:peer:2 (P2P Connection)
✅ Issuer: Invitation erstellt
   OOB ID: ce800ce6-ca19-4cae-a310-620090db48fc
   State: initial
   Invitation Msg ID: 2026c929-7288-42b9-8693-52023381c2c2
   Services: 1 service(s)
   Label: Issuer Agent
   Goal: Establish connection for credential issuance

✅ Holder: Invitation akzeptiert
   Connection ID (Holder): 06a88cfd-05b8-4ee7-b085-96ab3772c6ee

   Issuer sucht Connection via invitation_msg_id...
     Prüfe 1 Connection(s)...

   ✅ Issuer: Connection gefunden via invitation_msg_id!
      • Connection ID: 2c151234-7227-4adb-8758-7e72fb3b862a
      • State: active
      • Invitation Msg ID: 2026c929-7288-42b9-8693-52023381c2c2
      • Their Label: Holder Agent
      • Up

In [225]:
# Cell 15: Connection Holder → Verifier erstellen

print("🤝 Connection: Holder → Verifier erstellen...\n")

start_time = time.time()

# ========================================
# 1. PRE-CHECK: Existierende Connections prüfen
# ========================================
print("🔍 Pre-Check: Prüfe existierende Connections...\n")

existing_conn_verifier_holder = None
existing_conn_holder_verifier = None

# Prüfe Verifier-Seite: GET /connections
verifier_conns_response = api_get(VERIFIER_ADMIN_URL, "/connections")
if verifier_conns_response and "results" in verifier_conns_response:
    print(f"   Verifier: {len(verifier_conns_response['results'])} Connection(s) gefunden")
    for conn in verifier_conns_response["results"]:
        if conn.get("state") == "completed":
            existing_conn_verifier_holder = conn["connection_id"]
            print(f"     → Connection {existing_conn_verifier_holder} (State: completed)")
            break
else:
    print("   Verifier: Keine Connections gefunden")

# Prüfe Holder-Seite: GET /connections (für Holder→Verifier)
holder_conns_response = api_get(HOLDER_ADMIN_URL, "/connections")
if holder_conns_response and "results" in holder_conns_response:
    print(f"   Holder: {len(holder_conns_response['results'])} Connection(s) gefunden")
    # Finde Connection zum Verifier (basierend auf Label oder neueste)
    holder_verifier_conns = [c for c in holder_conns_response["results"] if c.get("their_label") == "Verifier Agent"]
    if holder_verifier_conns:
        for conn in holder_verifier_conns:
            if conn.get("state") == "completed":
                existing_conn_holder_verifier = conn["connection_id"]
                print(f"     → Connection {existing_conn_holder_verifier} (State: completed, Partner: Verifier)")
                break
else:
    print("   Holder: Keine Connections gefunden")

if existing_conn_verifier_holder and existing_conn_holder_verifier:
    print(f"\n✅ Existierende Connection gefunden!")
    print(f"   Verifier Connection ID: {existing_conn_verifier_holder}")
    print(f"   Holder Connection ID: {existing_conn_holder_verifier}")
    print("   Überspringe Connection-Erstellung\n")
    conn_id_verifier_holder = existing_conn_verifier_holder
    conn_id_holder_verifier = existing_conn_holder_verifier
else:
    print("\n   Keine vollständige existierende Connection gefunden")
    print("   Erstelle neue Connection mit did:peer:2...\n")
    
    # ========================================
    # 2. HAUPT-WORKFLOW: Connection mit did:peer erstellen
    # ========================================
    
    # Initialisiere Variablen
    conn_id_verifier_holder = None
    conn_id_holder_verifier = None
    invitation_msg_id = None
    invitation_key = None
    
    # Verifier erstellt Out-of-Band Invitation mit did:peer:2
    invitation_data = {
        "handshake_protocols": ["https://didcomm.org/didexchange/1.1"],
        "use_did_method": "did:peer:4",
        "my_label": "Verifier Agent",
        "goal": "Establish connection for credential verification",
        "goal_code": "verify-vc",
        "accept": [
            "didcomm/aip1",
            "didcomm/aip2;env=rfc19"
        ]
    }
    
    print("📋 Invitation-Modus: did:peer:2 (P2P Connection)")
    
    # API Call mit Query-Parametern für auto-accept
    invitation_response = api_post(
        VERIFIER_ADMIN_URL,
        "/out-of-band/create-invitation?auto_accept=true&multi_use=false",
        invitation_data
    )
    
    if invitation_response and "invitation" in invitation_response:
        invitation = invitation_response["invitation"]
        oob_id = invitation_response.get("oob_id")
        invitation_url = invitation_response.get("invitation_url", "")
        state = invitation_response.get("state")
        
        # WICHTIG: Speichere invitation_msg_id aus der Invitation
        invitation_msg_id = invitation.get("@id")
        
        # Optional: Extrahiere invitation_key aus services (falls vorhanden)
        services = invitation.get("services", [])
        if services and len(services) > 0:
            service = services[0]
            if isinstance(service, dict):
                recipient_keys = service.get("recipientKeys", [])
                if recipient_keys:
                    invitation_key = recipient_keys[0]
        
        print(f"✅ Verifier: Invitation erstellt")
        print(f"   OOB ID: {oob_id}")
        print(f"   State: {state}")
        print(f"   Invitation Msg ID: {invitation_msg_id}")
        if invitation_key:
            print(f"   Invitation Key: {invitation_key}")
        print(f"   Services: {len(invitation.get('services', []))} service(s)")
        print(f"   Label: {invitation_data['my_label']}")
        print(f"   Goal: {invitation_data['goal']}")
        
        # Holder akzeptiert Invitation (mit auto-accept)
        receive_response = api_post(
            HOLDER_ADMIN_URL,
            "/out-of-band/receive-invitation?auto_accept=true",
            invitation
        )
        
        if receive_response is not None:
            conn_id_holder_verifier = receive_response.get("connection_id")
            print(f"\n✅ Holder: Invitation akzeptiert")
            print(f"   Connection ID (Holder): {conn_id_holder_verifier}")
            
            # Verifier findet seine Connection via invitation_msg_id
            print(f"\n   Verifier sucht Connection via invitation_msg_id...")
            time.sleep(2)  # Kurze Wartezeit für DIDExchange Protocol
            
            # Hole alle Verifier Connections
            verifier_conns = api_get(VERIFIER_ADMIN_URL, "/connections")
            if verifier_conns and "results" in verifier_conns:
                connections = verifier_conns["results"]
                
                print(f"     Prüfe {len(connections)} Connection(s)...")
                
                # Finde Connection anhand invitation_msg_id (exaktes Match!)
                connection_found = False
                for conn in connections:
                    conn_invitation_msg_id = conn.get("invitation_msg_id")
                    conn_invitation_key = conn.get("invitation_key")
                    
                    # Match via invitation_msg_id (bevorzugt)
                    if invitation_msg_id and conn_invitation_msg_id == invitation_msg_id:
                        conn_id_verifier_holder = conn["connection_id"]
                        conn_state = conn.get("state")
                        print(f"\n   ✅ Verifier: Connection gefunden via invitation_msg_id!")
                        print(f"      • Connection ID: {conn_id_verifier_holder}")
                        print(f"      • State: {conn_state}")
                        print(f"      • Invitation Msg ID: {conn_invitation_msg_id}")
                        print(f"      • Their Label: {conn.get('their_label', 'N/A')}")
                        print(f"      • Updated At: {conn.get('updated_at', 'N/A')}")
                        connection_found = True
                        break
                    
                    # Fallback: Match via invitation_key
                    elif invitation_key and conn_invitation_key == invitation_key:
                        conn_id_verifier_holder = conn["connection_id"]
                        conn_state = conn.get("state")
                        print(f"\n   ✅ Verifier: Connection gefunden via invitation_key!")
                        print(f"      • Connection ID: {conn_id_verifier_holder}")
                        print(f"      • State: {conn_state}")
                        print(f"      • Invitation Key: {conn_invitation_key}...")
                        print(f"      • Their Label: {conn.get('their_label', 'N/A')}")
                        connection_found = True
                        break
                
                if not connection_found:
                    print(f"     ⚠️  Keine Connection mit matching invitation_msg_id gefunden")
                    print(f"     Gesuchte invitation_msg_id: {invitation_msg_id}")
            else:
                print(f"     ❌ Konnte Verifier Connections nicht abrufen")
        else:
            print("❌ Holder konnte Invitation nicht akzeptieren")
    else:
        print("❌ Fehler beim Erstellen der Invitation")

# ========================================
# 3. POST-VALIDATION: Connection-Status verifizieren
# ========================================
print("\n🔍 Post-Validation: Verifiziere Connection-Status...\n")

validation_success = False
verifier_state = None
holder_state = None

if conn_id_verifier_holder:
    # Prüfe Verifier-Seite mit spezifischem Endpoint: GET /connections/{conn_id}
    verifier_conn_detail = api_get(
        VERIFIER_ADMIN_URL,
        f"/connections/{conn_id_verifier_holder}"
    )
    
    if verifier_conn_detail:
        verifier_state = verifier_conn_detail.get("state")
        verifier_their_label = verifier_conn_detail.get("their_label")
        verifier_their_role = verifier_conn_detail.get("their_role")
        verifier_invitation_msg_id = verifier_conn_detail.get("invitation_msg_id")
        
        print(f"   📋 Verifier Connection {conn_id_verifier_holder}:")
        print(f"      • State: {verifier_state}")
        print(f"      • Their Label: {verifier_their_label}")
        print(f"      • Their Role: {verifier_their_role}")
        print(f"      • Invitation Msg ID: {verifier_invitation_msg_id}")

# Prüfe Holder-Seite (falls conn_id_holder_verifier bekannt)
if conn_id_holder_verifier:
    # Prüfe Holder-Seite mit spezifischem Endpoint: GET /connections/{conn_id}
    holder_conn_detail = api_get(
        HOLDER_ADMIN_URL,
        f"/connections/{conn_id_holder_verifier}"
    )
    
    if holder_conn_detail:
        holder_state = holder_conn_detail.get("state")
        holder_their_label = holder_conn_detail.get("their_label")
        holder_their_role = holder_conn_detail.get("their_role")
        holder_invitation_msg_id = holder_conn_detail.get("invitation_msg_id")
        
        print(f"\n   📋 Holder Connection {conn_id_holder_verifier}:")
        print(f"      • State: {holder_state}")
        print(f"      • Their Label: {holder_their_label}")
        print(f"      • Their Role: {holder_their_role}")
        print(f"      • Invitation Msg ID: {holder_invitation_msg_id}")
        
        # Verifiziere dass invitation_msg_id übereinstimmt
        if invitation_msg_id and verifier_invitation_msg_id == holder_invitation_msg_id:
            print(f"      ✅ Invitation Msg IDs stimmen überein!")

# Validierung
if verifier_state == "active" and holder_state == "active":
    validation_success = True
elif verifier_state == "active" and not conn_id_holder_verifier:
    # Bei existierender Connection nur Verifier-Seite bekannt
    validation_success = True

duration = time.time() - start_time
performance_metrics["connection_establishment"].append(duration)

# Finale Ausgabe
print(f"\n{'='*60}")
if validation_success:
    print(f"✅ CONNECTION ESTABLISHED (did:peer:2): Holder ↔ Verifier")
    print(f"   Verifier Connection ID: {conn_id_verifier_holder}")
    if conn_id_holder_verifier:
        print(f"   Holder Connection ID: {conn_id_holder_verifier}")
    print(f"   Verifier State: {verifier_state}")
    if holder_state:
        print(f"   Holder State: {holder_state}")
    if invitation_msg_id:
        print(f"   Invitation Msg ID: {invitation_msg_id}")
else:
    print(f"⚠️  CONNECTION STATUS UNCLEAR")
    print(f"   Verifier State: {verifier_state if verifier_state else 'N/A'}")
    print(f"   Holder State: {holder_state if holder_state else 'N/A'}")
    if conn_id_verifier_holder:
        print(f"   Verifier Connection ID: {conn_id_verifier_holder}")
    if conn_id_holder_verifier:
        print(f"   Holder Connection ID: {conn_id_holder_verifier}")

print(f"⏱️  Gesamtzeit: {duration:.3f}s")
print(f"{'='*60}")


🤝 Connection: Holder → Verifier erstellen...

🔍 Pre-Check: Prüfe existierende Connections...

   Verifier: 0 Connection(s) gefunden
   Holder: 1 Connection(s) gefunden

   Keine vollständige existierende Connection gefunden
   Erstelle neue Connection mit did:peer:2...

📋 Invitation-Modus: did:peer:2 (P2P Connection)
✅ Verifier: Invitation erstellt
   OOB ID: 9322d867-b829-4801-b209-09e3e7ce1337
   State: initial
   Invitation Msg ID: 3616fb40-fa18-4994-b12e-7d429d2177da
   Services: 1 service(s)
   Label: Verifier Agent
   Goal: Establish connection for credential verification

✅ Holder: Invitation akzeptiert
   Connection ID (Holder): c4abd96d-e153-4118-91b4-18ef6bd81e1c

   Verifier sucht Connection via invitation_msg_id...
     Prüfe 1 Connection(s)...

   ✅ Verifier: Connection gefunden via invitation_msg_id!
      • Connection ID: 42e38550-dc62-4d8d-8bda-b5e3a236ab0a
      • State: active
      • Invitation Msg ID: 3616fb40-fa18-4994-b12e-7d429d2177da
      • Their Label: Holder 

In [226]:
# Cell 16: Connection-Übersicht anzeigen

print("📊 Connection-Übersicht:\n")

connections_overview = []

# Issuer Connections
issuer_conns = api_get(ISSUER_ADMIN_URL, "/connections")
if issuer_conns and "results" in issuer_conns:
    for conn in issuer_conns["results"]:
        connections_overview.append({
            "Agent": "Issuer",
            "Partner": conn.get("their_label", "Unknown"),
            "Connection ID": conn["connection_id"],
            "State": conn["state"],
            "Invitation Msg ID": conn.get("invitation_msg_id", "N/A") if conn.get("invitation_msg_id") else "N/A",
            "Status": "✅" if conn["state"] == "active" else "⏳"
        })

# Holder Connections
holder_conns = api_get(HOLDER_ADMIN_URL, "/connections")
if holder_conns and "results" in holder_conns:
    for conn in holder_conns["results"]:
        partner = conn.get("their_label", "Unknown")
        connections_overview.append({
            "Agent": "Holder",
            "Partner": partner,
            "Connection ID": conn["connection_id"],
            "State": conn["state"],
            "Invitation Msg ID": conn.get("invitation_msg_id", "N/A") if conn.get("invitation_msg_id") else "N/A",
            "Status": "✅" if conn["state"] == "active" else "⏳"
        })

# Verifier Connections
verifier_conns = api_get(VERIFIER_ADMIN_URL, "/connections")
if verifier_conns and "results" in verifier_conns:
    for conn in verifier_conns["results"]:
        connections_overview.append({
            "Agent": "Verifier",
            "Partner": conn.get("their_label", "Unknown"),
            "Connection ID": conn["connection_id"],
            "State": conn["state"],
            "Invitation Msg ID": conn.get("invitation_msg_id", "N/A") if conn.get("invitation_msg_id") else "N/A",
            "Status": "✅" if conn["state"] == "active" else "⏳"
        })

df_connections = pd.DataFrame(connections_overview)
display(df_connections)

# Detaillierte Connection-Informationen
print("\n📋 Detaillierte Connection-Informationen:\n")

# Gruppiere nach invitation_msg_id um zusammengehörige Connections zu finden
all_connections = []
if issuer_conns and "results" in issuer_conns:
    for conn in issuer_conns["results"]:
        all_connections.append(("Issuer", conn))
if holder_conns and "results" in holder_conns:
    for conn in holder_conns["results"]:
        all_connections.append(("Holder", conn))
if verifier_conns and "results" in verifier_conns:
    for conn in verifier_conns["results"]:
        all_connections.append(("Verifier", conn))

# Gruppiere nach invitation_msg_id
from collections import defaultdict
connection_groups = defaultdict(list)
for agent, conn in all_connections:
    invitation_msg_id = conn.get("invitation_msg_id")
    if invitation_msg_id:
        connection_groups[invitation_msg_id].append((agent, conn))

# Zeige gruppierte Connections
for idx, (inv_msg_id, group) in enumerate(connection_groups.items(), 1):
    print(f"Connection Group {idx}:")
    print(f"   Invitation Msg ID: {inv_msg_id}")
    for agent, conn in group:
        print(f"   • {agent}: {conn['connection_id']} (State: {conn['state']})")
    print()


📊 Connection-Übersicht:



,Agent,Partner,Connection ID,State,Invitation Msg ID,Status
0,Issuer,Holder Agent,2c151234-7227-4adb-8758-7e72fb3b862a,active,2026c929-7288-42b9-8693-52023381c2c2,✅
1,Holder,Issuer Agent,06a88cfd-05b8-4ee7-b085-96ab3772c6ee,active,2026c929-7288-42b9-8693-52023381c2c2,✅
2,Holder,Verifier Agent,c4abd96d-e153-4118-91b4-18ef6bd81e1c,active,3616fb40-fa18-4994-b12e-7d429d2177da,✅
3,Verifier,Holder Agent,42e38550-dc62-4d8d-8bda-b5e3a236ab0a,active,3616fb40-fa18-4994-b12e-7d429d2177da,✅



📋 Detaillierte Connection-Informationen:

Connection Group 1:
   Invitation Msg ID: 2026c929-7288-42b9-8693-52023381c2c2
   • Issuer: 2c151234-7227-4adb-8758-7e72fb3b862a (State: active)
   • Holder: 06a88cfd-05b8-4ee7-b085-96ab3772c6ee (State: active)

Connection Group 2:
   Invitation Msg ID: 3616fb40-fa18-4994-b12e-7d429d2177da
   • Holder: c4abd96d-e153-4118-91b4-18ef6bd81e1c (State: active)
   • Verifier: 42e38550-dc62-4d8d-8bda-b5e3a236ab0a (State: active)



In [227]:
# Cell 17 Wallet DID Übersicht anzeigen

print("📋 Wallet DID Übersicht - Alle DIDs:\n")

# Alle DIDs von allen Agenten abrufen
issuer_dids = api_get(ISSUER_ADMIN_URL, "/wallet/did")
holder_dids = api_get(HOLDER_ADMIN_URL, "/wallet/did")
verifier_dids = api_get(VERIFIER_ADMIN_URL, "/wallet/did")

# Zeige originale Responses
print("🔍 Originale /wallet/did Responses:\n")
if issuer_dids:
    pretty_print(issuer_dids, "Issuer Wallet DIDs")
if holder_dids:
    pretty_print(holder_dids, "Holder Wallet DIDs")
if verifier_dids:
    pretty_print(verifier_dids, "Verifier Wallet DIDs")

# Erstelle erweiterte Übersicht
print("\n📊 DID-Übersicht (Detailliert):\n")
ledger_overview = []

# Issuer DIDs
if issuer_dids and "results" in issuer_dids:
    for did_info in issuer_dids["results"]:
        posture = did_info.get("posture", "unknown")
        # Zeige alle DIDs (posted und wallet_only)
        ledger_overview.append({
            "Agent": "Issuer",
            "DID": did_info["did"],
            "Verkey": did_info.get("verkey", "N/A"),
            "Posture": posture,
            "Method": did_info.get("method", "sov"),
            "Key Type": did_info.get("key_type", "N/A"),
            "On Ledger": "✅" if posture in ["public", "posted"] else "❌",
            "Endpoint": did_info.get("metadata", {}).get("endpoint", "N/A")
        })

# Holder DIDs
if holder_dids and "results" in holder_dids:
    for did_info in holder_dids["results"]:
        posture = did_info.get("posture", "unknown")
        ledger_overview.append({
            "Agent": "Holder",
            "DID": did_info["did"],
            "Verkey": did_info.get("verkey", "N/A"),
            "Posture": posture,
            "Method": did_info.get("method", "sov"),
            "Key Type": did_info.get("key_type", "N/A"),
            "On Ledger": "✅" if posture in ["public", "posted"] else "❌",
            "Endpoint": did_info.get("metadata", {}).get("endpoint", "N/A")
        })

# Verifier DIDs
if verifier_dids and "results" in verifier_dids:
    for did_info in verifier_dids["results"]:
        posture = did_info.get("posture", "unknown")
        ledger_overview.append({
            "Agent": "Verifier",
            "DID": did_info["did"],
            "Verkey": did_info.get("verkey", "N/A"),
            "Posture": posture,
            "Method": did_info.get("method", "sov"),
            "Key Type": did_info.get("key_type", "N/A"),
            "On Ledger": "✅" if posture in ["public", "posted"] else "❌",
            "Endpoint": did_info.get("metadata", {}).get("endpoint", "N/A")
        })

# Als DataFrame anzeigen
if ledger_overview:
    df_ledger = pd.DataFrame(ledger_overview)
    display(df_ledger)
else:
    print("⚠️  Keine DIDs gefunden")

# Zusammenfassung
print("\n💡 Posture-Bedeutung:")
print("   • posted       = DID auf Ledger registriert und veröffentlicht")
print("   • public       = DID als Public DID gesetzt")
print("   • wallet_only  = DID nur im Wallet, nicht auf Ledger")
print("\n💡 Hinweis:")
print("   - Issuer & Verifier benötigen Public DIDs für Schema/CredDef")
print("   - Holder nutzt nur Peer-DIDs für sichere P2P-Verbindungen")
print("   - Alte DIDs (wallet_only) können aus vergangenen Runs stammen")

📋 Wallet DID Übersicht - Alle DIDs:

🔍 Originale /wallet/did Responses:


  Issuer Wallet DIDs
{
  "results": [
    {
      "did": "did:indy:9pbXiFBZZGwXKp61HQBz3J",
      "verkey": "2zoa6G7aMfX8GnUEpDxxunFHE7fZktRiiHk1vgMRH2tm",
      "posture": "posted",
      "key_type": "ed25519",
      "method": "indy",
      "metadata": {
        "posted": true,
        "endpoint": "https://host.docker.internal:8020"
      }
    },
    {
      "did": "did:peer:4zQmbeBCb7YRyPNyEWF8CyxPGxDL9AQNrwvYpXTUVTkMSshh",
      "verkey": "6Y5NTJ3JvoLHRbivZdCtSswJrhe37FRQSG2pXm4SFhVZzQNvpWrk5DeBkryeTjMeEcWBZyqMhUGoc9DexwkHv1noAeRAfdHfMA58FR3PCmACYGz3C8w9ssc7wDBSwyBNBhWJeFsL8Hr6HoAkz7bxbiy4mET4crazZN8oPDpe6TYZUf3RAwfUvxe2QEYQBUjevkxxbidczAg9zUQmNWDbwnGV9Adkhb3Z6nVt7oha1n5r1qv6ab7s38FBZ8eHc7EWCN6JfaicLhGNeywg86YnjYW7X1TjodEAQTq2ojNMGEQNRgowMeZkNDM13Xo9jtt5G1vwkp7MrKNn8WygafMR1VR2sFNL1Ze9pur5F6ndFY4rVNDsMrAZAswgsVGTtVoLSWHKXkNuYpee1KdKC6Xp8aPbbxjjzVoNv6FesK6JqLHY1VyDWiujEh9AnGqbEedbSzPxpXzdLMsuNdbp8eeKGNDR47q53d

,Agent,DID,Verkey,Posture,Method,Key Type,On Ledger,Endpoint
0,Issuer,did:indy:9pbXiFBZZGwXKp61HQBz3J,2zoa6G7aMfX8GnUEpDxxunFHE7fZktRiiHk1vgMRH2tm,posted,indy,ed25519,✅,https://host.docker.internal:8020
1,Issuer,did:peer:4zQmbeBCb7YRyPNyEWF8CyxPGxDL9AQNrwvYp...,6Y5NTJ3JvoLHRbivZdCtSswJrhe37FRQSG2pXm4SFhVZzQ...,wallet_only,did:peer:4,ml-dsa-65,❌,N/A
2,Issuer,did:peer:4zQmbeBCb7YRyPNyEWF8CyxPGxDL9AQNrwvYp...,6Y5NTJ3JvoLHRbivZdCtSswJrhe37FRQSG2pXm4SFhVZzQ...,wallet_only,did:peer:4,ml-dsa-65,❌,N/A
3,Issuer,did:peer:4zQmeJPv2J866iYB8gsMHBZxUfbvHuByDYT1w...,Ff9aJ7EXepQszVnegcZVq5yK11RLvrUCTMEUF3knK1A31v...,wallet_only,did:peer:4,ml-dsa-65,❌,N/A
4,Holder,did:peer:4zQmXESDvTb2RBHKbFp7YjS7qqx6KNegSZ6vD...,7jFycjmhWR4uZuy8CZ9BkyVnWkotATaiTwFzPQjs5bPkUq...,wallet_only,did:peer:4,ml-dsa-65,❌,N/A
5,Holder,did:peer:4zQmXESDvTb2RBHKbFp7YjS7qqx6KNegSZ6vD...,7jFycjmhWR4uZuy8CZ9BkyVnWkotATaiTwFzPQjs5bPkUq...,wallet_only,did:peer:4,ml-dsa-65,❌,N/A
6,Holder,did:peer:4zQmd46UnTb6BArenhZLqx1Ux2JmtkGJPy23z...,HyNoMHUScgKdZGWe3FBZzjDxKrboviex8s4B5MPpnyQtdQ...,wallet_only,did:peer:4,ml-dsa-65,❌,N/A
7,Holder,did:peer:4zQmd46UnTb6BArenhZLqx1Ux2JmtkGJPy23z...,HyNoMHUScgKdZGWe3FBZzjDxKrboviex8s4B5MPpnyQtdQ...,wallet_only,did:peer:4,ml-dsa-65,❌,N/A
8,Verifier,did:indy:DuLeEuX4wg3EFiEmtz6fcU,8eXpYXH2tQRvkHv5XXX8r5nCS4CabRP6i2gnyeSzEEFL,posted,indy,ed25519,✅,https://host.docker.internal:8040
9,Verifier,did:peer:4zQmdK9nSFrMkHBiGnXFiYzQoMmVgPbh88Dus...,Bz6RAqMQZmHhH9esutxJteC4Zxc5NNBYHVabSAtns48ZQZ...,wallet_only,did:peer:4,ml-dsa-65,❌,N/A



💡 Posture-Bedeutung:
   • posted       = DID auf Ledger registriert und veröffentlicht
   • public       = DID als Public DID gesetzt
   • wallet_only  = DID nur im Wallet, nicht auf Ledger

💡 Hinweis:
   - Issuer & Verifier benötigen Public DIDs für Schema/CredDef
   - Holder nutzt nur Peer-DIDs für sichere P2P-Verbindungen
   - Alte DIDs (wallet_only) können aus vergangenen Runs stammen


## Teil 5: Credential Issuance - KRITIS-Notfall-Wartungszertifikat

**Szenario**: Ein Energienetzbetreiber (Issuer) stellt einem Notfalltechniker (Holder) ein zeitlich begrenztes **Notfall-Wartungszertifikat** aus, das den Zugang zu kritischer Infrastruktur (Umspannwerk Nord-Ost) autorisiert.

**KRITIS-Kontext**:
- Kritische Infrastruktur gemäß EU-Verordnung 2019/881 (Cybersecurity Act)
- Energiesektor (Stromversorgung) - eine der 9 KRITIS-Branchen
- Zero-Knowledge-Proof-basierte Zutrittskontrolle ohne Preisgabe sensibler Daten
- Post-Quantum-Kryptographie (ML-DSA-65) für langfristige Sicherheit

**Credential-Attribute**:
- **Identität**: Vorname, Nachname, Organisation, Rolle
- **Berechtigung**: Zertifikatstyp, Anlagentyp, Sicherheitsstufe
- **Zeitfenster**: Epoch-basierte Gültigkeitszeiträume (Unix Timestamps)

In [228]:
# Cell 18: Credential Offer senden (Issuer → Holder)

print("📤 Credential Offer senden (Issuer → Holder)...\n")

start_time = time.time()

# Credential-Daten vorbereiten
credential_offer_data = {
    "comment": "KRITIS Notfall-Wartungszertifikat",
    "connection_id": conn_id_issuer_holder,
    "filter": {
        "indy": {
            "cred_def_id": cred_def_id
        }
    },
    "credential_preview": {
        "@type": "issue-credential/2.0/credential-preview",
        "attributes": [
            {"name": "first_name", "value": "Max"},
            {"name": "name", "value": "Mustermann"},
            {"name": "organisation", "value": "Musterfirma GmbH"},
            {"name": "role", "value": "Notfalltechniker"},
            {"name": "cert_type", "value": "Notfall-Wartungsberechtigung"},
            {"name": "facility_type", "value": "Umspannwerk Nord-Ost"},
            {"name": "epoch_valid_from", "value": "1765026000"},
            {"name": "epoch_valid_until", "value": "1765033200"},
            {"name": "security_access_level", "value": "Stufe-3-Kritisch"}
        ]
    },
    "trace": False
}

# Credential Offer senden
cred_offer_response = api_post(
    ISSUER_ADMIN_URL,
    "/issue-credential-2.0/send-offer",
    credential_offer_data
)

if cred_offer_response is not None:
    cred_ex_id = cred_offer_response.get("cred_ex_id")
    cred_ex_state = cred_offer_response.get("state")

    print(f"✅ Credential Offer gesendet")
    print(f"   Exchange ID: {cred_ex_id}")
    print(f"   State:       {cred_ex_state}")

    # ========================================
    # WARTE AUF CREDENTIAL-AUSSTELLUNG
    # ========================================
    print("\n   ⏰ Warte auf Credential-Ausstellung (auto-store auf Holder)...")
    time.sleep(5)

    # ========================================
    # STATUS-CHECK AUF BEIDEN SEITEN
    # ========================================
    print("\n🔍 Status-Check:\n")

    # 1. Issuer-Seite Status (BENÖTIGT für Revocation IDs!)
    cred_status_issuer = api_get(
        ISSUER_ADMIN_URL,
        f"/issue-credential-2.0/records/{cred_ex_id}"
    )

    preserve_flag_active = False
    issuer_state = None

    if cred_status_issuer:
        preserve_flag_active = True
        issuer_state = cred_status_issuer.get("cred_ex_record", {}).get("state")
        print(f"   📋 Issuer Exchange:")
        print(f"      • State: {issuer_state}")
        print(f"      ✅ --preserve-exchange-records aktiv!")
    else:
        print(f"   ⚠️  Issuer Exchange nicht abrufbar")
        print(f"      ❌ --preserve-exchange-records ist NICHT aktiv!")

    # 2. Holder Credentials prüfen
    print("\n   📋 Holder Credentials:")
    holder_creds = api_get(HOLDER_ADMIN_URL, "/credentials")

    credential_stored = False
    credential_referent = None

    if holder_creds and "results" in holder_creds:
        for cred in holder_creds["results"]:
            if cred.get("cred_def_id") == cred_def_id:
                credential_stored = True
                credential_referent = cred.get("referent")
                print(f"      • Credential ID: {credential_referent}")
                print(f"      • Schema ID: {cred.get('schema_id', 'N/A')[:50]}...")

                # Zeige Attribute
                attrs = cred.get("attrs", {})
                if attrs:
                    print(f"      • Attributes:")
                    for key, value in list(attrs.items())[:3]:
                        print(f"         - {key}: {value}")
                    if len(attrs) > 3:
                        print(f"         ... und {len(attrs)-3} weitere")
                break

    # ========================================
    # REVOCATION IDs SPEICHERN (KRITISCH!)
    # ========================================
    rev_reg_id = None
    cred_rev_id = None

    if cred_status_issuer:
        indy_data = cred_status_issuer.get("indy", {})
        rev_reg_id = indy_data.get("rev_reg_id")
        cred_rev_id = indy_data.get("cred_rev_id")

        if rev_reg_id and cred_rev_id:
            print("\n" + "="*60)
            print("🔑 REVOCATION-INFORMATIONEN")
            print("="*60)
            print(f"   • Revocation Registry ID: {rev_reg_id}")
            print(f"   • Credential Revocation ID: {cred_rev_id}")
            print("   💾 IDs gespeichert für Revocation-Workflow (Cell 32-33)")
            print("="*60 + "\n")

    # ========================================
    # ISSUER CREDENTIAL REGISTRY
    # ========================================
    print("="*60)
    print("📋 ISSUER: Alle ausgestellten Credentials")
    print("="*60)

    all_issuer_records = api_get(ISSUER_ADMIN_URL, "/issue-credential-2.0/records")

    if all_issuer_records and "results" in all_issuer_records:
        results = all_issuer_records["results"]

        if len(results) > 0:
            print(f"\n✅ Issuer hat {len(results)} Credential Exchange Record(s)\n")

            for idx, record in enumerate(results, 1):
                print(f"   Credential #{idx}:")
                print(f"      • Exchange ID: {record['cred_ex_record']['cred_ex_id']}")
                print(f"      • State: {record['cred_ex_record']['state']}")

                # Cred Def ID
                cred_def_from_record = None
                if "cred_offer" in record:
                    cred_def_from_record = record.get("cred_offer", {}).get("cred_def_id")

                if cred_def_from_record:
                    print(f"      • Cred Def ID: {cred_def_from_record[:50]}...")

                # Attributes aus Preview
                if "cred_preview" in record:
                    attrs = record.get("cred_preview", {}).get("attributes", [])
                    if attrs:
                        print(f"      • Attributes:")
                        for attr in attrs[:3]:
                            print(f"         - {attr.get('name')}: {attr.get('value')}")
                        if len(attrs) > 3:
                            print(f"         ... und {len(attrs)-3} weitere")
                print()
        else:
            print("\n⚠️  Registry ist LEER\n")

            if not preserve_flag_active:
                print("   ❌ --preserve-exchange-records ist NICHT aktiv!")
                print("   🔧 Lösung: docker-compose restart issuer")
            else:
                print("   💡 Credential erscheint nach erneutem Ausführen dieser Cell")
            print()

    # ========================================
    # FINALE SUMMARY
    # ========================================
    duration = time.time() - start_time
    performance_metrics["credential_issuance"].append(duration)

    print("="*60)
    if credential_stored:
        print(f"✅ KRITIS-NOTFALL-WARTUNGSZERTIFIKAT ERFOLGREICH AUSGESTELLT")
        print(f"   Issuer State: {issuer_state if issuer_state else 'done'}")
        print(f"   Credential im Holder Wallet: ✅")
        print(f"   Credential Referent: {credential_referent}")

        if rev_reg_id and cred_rev_id:
            print(f"   Revocation IDs gespeichert: ✅")
        else:
            print(f"   ⚠️  Revocation IDs nicht gefunden!")

        if preserve_flag_active:
            print(f"   Issuer Registry verfügbar: ✅")
        else:
            print(f"   Issuer Registry verfügbar: ⚠️  (restart issuer)")
    else:
        print(f"⚠️  CREDENTIAL STATUS UNKLAR")
        print(f"   Issuer State: {issuer_state if issuer_state else 'N/A'}")
        print(f"   Credential im Wallet: ❌")

    print(f"⏱️  Gesamtzeit: {duration:.3f}s")
    print(f"{'='*60}")

    # Zeige vollständige Response
    # pretty_print(cred_status_issuer, "Credential Offer Response (KRITIS)")

else:
    print("❌ Fehler beim Senden des Credential Offers")

📤 Credential Offer senden (Issuer → Holder)...

✅ Credential Offer gesendet
   Exchange ID: 67e0d17f-3de5-4dc8-8c20-927de9393bb1
   State:       offer-sent

   ⏰ Warte auf Credential-Ausstellung (auto-store auf Holder)...

🔍 Status-Check:

   📋 Issuer Exchange:
      • State: done
      ✅ --preserve-exchange-records aktiv!

   📋 Holder Credentials:
      • Credential ID: 673556ae-53fd-47a6-aaa9-f22d087814ca
      • Schema ID: 9pbXiFBZZGwXKp61HQBz3J:2:kritis_emergency_maintena...
      • Attributes:
         - epoch_valid_from: 1765026000
         - cert_type: Notfall-Wartungsberechtigung
         - name: Mustermann
         ... und 6 weitere

🔑 REVOCATION-INFORMATIONEN
   • Revocation Registry ID: 9pbXiFBZZGwXKp61HQBz3J:4:9pbXiFBZZGwXKp61HQBz3J:3:CL:10:default:CL_ACCUM:063f8bc9-be4e-4d93-9155-ceb928c63b60
   • Credential Revocation ID: 1
   💾 IDs gespeichert für Revocation-Workflow (Cell 32-33)

📋 ISSUER: Alle ausgestellten Credentials

✅ Issuer hat 1 Credential Exchange Record(s)

   

In [229]:
# Cell 19: Holder Credentials im Detail anzeigen

print("🎓 Holder Wallet - Gespeicherte Credentials:\n")

holder_creds = api_get(HOLDER_ADMIN_URL, "/credentials")

if holder_creds and "results" in holder_creds and len(holder_creds["results"]) > 0:
    for idx, cred in enumerate(holder_creds["results"], 1):
        print(f"\n{'='*60}")
        print(f"Credential #{idx}")
        print(f"{'='*60}")
        print(f"Referent: {cred.get('referent', 'N/A')}")
        print(f"Schema ID: {cred.get('schema_id', 'N/A')}")
        print(f"Cred Def ID: {cred.get('cred_def_id', 'N/A')}")
        
        # Credentials Revoked Abfrage gegen /credential/revoked/{credential_id ==> Referent}
        print(f"Revoked Status: {api_get(HOLDER_ADMIN_URL, f'/credential/revoked/{cred.get('referent', 'N/A')}').get('revoked', 'N/A')}")

        if "attrs" in cred:
            print("\nAttribute:")
            for attr_name, attr_value in cred["attrs"].items():
                print(f"  - {attr_name}: {attr_value}")
        
        
    
    # Als DataFrame anzeigen
    creds_list = []
    for cred in holder_creds["results"]:
        if "attrs" in cred:
            cred_info = {"Credential ID": cred.get("referent", "N/A")}
            cred_info.update(cred["attrs"])
            creds_list.append(cred_info)
    
    if creds_list:
        print("\n📊 Credential-Übersicht:")
        df_creds = pd.DataFrame(creds_list)
        display(df_creds)
else:
    print("⚠️  Holder hat noch keine Credentials")

🎓 Holder Wallet - Gespeicherte Credentials:


Credential #1
Referent: 673556ae-53fd-47a6-aaa9-f22d087814ca
Schema ID: 9pbXiFBZZGwXKp61HQBz3J:2:kritis_emergency_maintenance_cert:1.0
Cred Def ID: 9pbXiFBZZGwXKp61HQBz3J:3:CL:10:default
Revoked Status: False

Attribute:
  - epoch_valid_until: 1765033200
  - first_name: Max
  - name: Mustermann
  - role: Notfalltechniker
  - facility_type: Umspannwerk Nord-Ost
  - epoch_valid_from: 1765026000
  - security_access_level: Stufe-3-Kritisch
  - organisation: Musterfirma GmbH
  - cert_type: Notfall-Wartungsberechtigung

📊 Credential-Übersicht:


,Credential ID,epoch_valid_until,first_name,name,role,facility_type,epoch_valid_from,security_access_level,organisation,cert_type
0,673556ae-53fd-47a6-aaa9-f22d087814ca,1765033200,Max,Mustermann,Notfalltechniker,Umspannwerk Nord-Ost,1765026000,Stufe-3-Kritisch,Musterfirma GmbH,Notfall-Wartungsberechtigung


## Teil 6: Proof Presentation - Zutrittskontrolle am Umspannwerk

**Szenario**: Der Notfalltechniker (Holder) präsentiert sein Credential am Zutrittssystem des Umspannwerks (Verifier). Das System fordert einen Zero-Knowledge-Proof an, der folgende Eigenschaften verifiziert:

1. **Berechtigung (REVEALED)**: Zertifikatstyp, Anlagentyp, Sicherheitsstufe werden offengelegt
2. **Gültigkeitszeitraum (REVEALED)**: Epoch-Timestamps (valid_from, valid_until) werden offengelegt
3. **Rolle (REVEALED)**: Berufliche Funktion wird offengelegt (z.B. "Notfalltechniker")
4. **Revocation-Check**: Credential ist NICHT revoked (Revocation Registry Check)
5. **Identität (UNREVEALED)**: Vor- und Nachname bleiben GEHEIM (Zero-Knowledge-Proof)
6. **Arbeitgeber (UNREVEALED)**: Organisation bleibt GEHEIM (Zero-Knowledge-Proof)

**Datenschutz-Mechanismus (Privacy by Design)**:
- **Selective Disclosure**: Nur 6 von 9 Attributen werden offengelegt (cert_type, facility_type, security_access_level, epoch_valid_from, epoch_valid_until, role)
- **Zero-Knowledge-Proof für Identität**: Das Zutrittssystem erfährt NICHT, WER der Techniker ist (Vor-/Nachname geheim)
- **Zero-Knowledge-Proof für Arbeitgeber**: Das Zutrittssystem erfährt NICHT, für WELCHE Firma der Techniker arbeitet
- **ZKP-Beweis**: Der Holder beweist kryptographisch, dass er ein gültiges Credential mit korrekten Berechtigungen besitzt, ohne seine Identität offenzulegen
- **DSGVO-Konformität**: Minimierung personenbezogener Daten (Art. 5 Abs. 1 lit. c DSGVO) - Pseudonymisierung durch ZKP

**KRITIS-Sicherheitsmechanismus**:
- **Revocation Always-ON**: Echtzeit-Prüfung gegen Hyperledger Indy Revocation Registry
- **Non-Revocation Interval**: `{"from": 0, "to": current_timestamp}` - Credential darf zu KEINEM Zeitpunkt revoked worden sein
- **PQC-Signaturen**: Proof wird mit ML-DSA-65 signiert (quantensicher)

In [244]:
# Cell 20: Proof Request senden (Verifier → Holder) - mit REVOCATION ALWAYS-ON

print("🔍 Proof Request senden (Verifier → Holder)...\n")

start_time = time.time()

# Get current timestamp for non-revocation interval
current_timestamp = int(time.time())

# Proof Request Data (ACA-Py API) - Privacy-Preserving ZKP!
# API: POST /present-proof-2.0/send-request
# REVEALED: cert_type, facility_type, security_access_level, epoch_valid_from, epoch_valid_until, role (6 Attribute)
# UNREVEALED: first_name, name, organisation (3 Attribute - ZKP ohne Offenlegung!)
proof_request_data = {
    "connection_id": conn_id_verifier_holder,  # from Cell 15
    "presentation_request": {
        "indy": {
            "name": "Proof of KRITIS Emergency Maintenance Authorization",
            "version": "1.0",
            "requested_attributes": {
                "attr1_referent": {
                    "name": "cert_type",
                    "restrictions": [{"cred_def_id": cred_def_id}],
                    "non_revoked": {"from": 0, "to": current_timestamp}
                },
                "attr2_referent": {
                    "name": "facility_type",
                    "restrictions": [{"cred_def_id": cred_def_id}],
                    "non_revoked": {"from": 0, "to": current_timestamp}
                },
                "attr3_referent": {
                    "name": "security_access_level",
                    "restrictions": [{"cred_def_id": cred_def_id}],
                    "non_revoked": {"from": 0, "to": current_timestamp}
                },
                "attr4_referent": {
                    "name": "epoch_valid_from",
                    "restrictions": [{"cred_def_id": cred_def_id}],
                    "non_revoked": {"from": 0, "to": current_timestamp}
                },
                "attr5_referent": {
                    "name": "epoch_valid_until",
                    "restrictions": [{"cred_def_id": cred_def_id}],
                    "non_revoked": {"from": 0, "to": current_timestamp}
                },
                "attr6_referent": {
                    "name": "role",
                    "restrictions": [{"cred_def_id": cred_def_id}],
                    "non_revoked": {"from": 0, "to": current_timestamp}
                }
            },
            "requested_predicates": {}
        }
    }
}

print(f"🔍 Proof Request für KRITIS-Zutrittsberechtigung:")
print(f"   Endpoint:      /present-proof-2.0/send-request")
print(f"   Connection:    {conn_id_verifier_holder[:20]}...")
print(f"   Cred Def ID:   {cred_def_id[:40]}...")
print(f"   Proof Name:    Proof of KRITIS Emergency Maintenance Authorization")
print(f"   REVEALED:      Berechtigung, Anlage, Sicherheitsstufe, Gültigkeitszeitraum, Rolle (6 Attribute)")
print(f"   ⚠️  DATENSCHUTZ: Identität (Vor-/Nachname) und Organisation werden NICHT offengelegt (Zero-Knowledge-Proof)\n")

# Send Proof Request (ACA-Py endpoint)
proof_request_response = api_post(
    VERIFIER_ADMIN_URL,
    "/present-proof-2.0/send-request",
    proof_request_data
)

if proof_request_response is not None:
    # Response Format: {"pres_ex_id": "...", "state": "...", ...}
    pres_ex_id = proof_request_response.get("pres_ex_id")
    pres_ex_state = proof_request_response.get("state")

    duration = time.time() - start_time
    performance_metrics["proof_request"].append(duration)

    print(f"✅ Proof Request gesendet")
    print(f"   Exchange ID: {pres_ex_id}")
    print(f"   State:       {pres_ex_state}")
    print(f"⏱️  Zeit: {duration:.3f}s")

    # Show full Proof Request Response
    pretty_print(proof_request_response, "Proof Request Response (KRITIS)")

else:
    print("❌ Fehler beim Senden des Proof Requests")

🔍 Proof Request senden (Verifier → Holder)...

🔍 Proof Request für KRITIS-Zutrittsberechtigung:
   Endpoint:      /present-proof-2.0/send-request
   Connection:    42e38550-dc62-4d8d-8...
   Cred Def ID:   9pbXiFBZZGwXKp61HQBz3J:3:CL:10:default...
   Proof Name:    Proof of KRITIS Emergency Maintenance Authorization
   REVEALED:      Berechtigung, Anlage, Sicherheitsstufe, Gültigkeitszeitraum, Rolle (6 Attribute)
   ⚠️  DATENSCHUTZ: Identität (Vor-/Nachname) und Organisation werden NICHT offengelegt (Zero-Knowledge-Proof)

✅ Proof Request gesendet
   Exchange ID: f1a25300-cad9-411b-b060-c638e977b9ee
   State:       request-sent
⏱️  Zeit: 0.032s

  Proof Request Response (KRITIS)
{
  "state": "request-sent",
  "created_at": "2025-12-07T17:57:28.475846Z",
  "updated_at": "2025-12-07T17:57:28.475846Z",
  "trace": false,
  "pres_ex_id": "f1a25300-cad9-411b-b060-c638e977b9ee",
  "connection_id": "42e38550-dc62-4d8d-8bda-b5e3a236ab0a",
  "thread_id": "1a5e91cf-02d7-47f0-bd2e-b08e6b03e21f",
 

In [245]:
# Cell 21: Holder - Proof Request empfangen und Credentials auswählen

print("📥 Holder: Proof Request verarbeiten...\n")

if pres_ex_id:
    # Holder prüft empfangene Proof Requests
    time.sleep(2)
    
    holder_pres_ex = api_get(HOLDER_ADMIN_URL, "/present-proof-2.0/records")
    
    if holder_pres_ex and "results" in holder_pres_ex:
        # Finde den aktuellen Presentation Exchange
        current_pres = None
        for pres in holder_pres_ex["results"]:
            if pres.get("state") == "request-received":
                current_pres = pres
                holder_pres_ex_id = pres["pres_ex_id"]
                break
        
        if current_pres:
            print(f"✅ Holder hat Proof Request empfangen")
            print(f"   Presentation Exchange ID: {holder_pres_ex_id}")
            
            # Credentials für die Presentation abrufen
            creds_for_pres = api_get(
                HOLDER_ADMIN_URL,
                f"/present-proof-2.0/records/{holder_pres_ex_id}/credentials"
            )
            
            if creds_for_pres:
                print(f"\n📋 Verfügbare Credentials für Presentation:")
                
                # API kann entweder list oder dict zurückgeben
                total_creds = 0
                
                if isinstance(creds_for_pres, list):
                    # Liste von Credential Records
                    total_creds = len(creds_for_pres)
                    print(f"   Gefunden: {total_creds} Credential(s)")
                    
                    # Zeige Details der ersten paar Credentials
                    for idx, cred in enumerate(creds_for_pres[:3], 1):
                        print(f"\n   Credential #{idx}:")
                        if "cred_info" in cred:
                            cred_info = cred["cred_info"]
                            print(f"      • Referent: {cred_info.get('referent', 'N/A')}")
                            print(f"      • Schema ID: {cred_info.get('schema_id', 'N/A')}")
                            print(f"      • Cred Def ID: {cred_info.get('cred_def_id', 'N/A')}")
                            
                            # Zeige Attribute
                            attrs = cred_info.get("attrs", {})
                            if attrs:
                                print(f"      • Attributes:")
                                # Zeige erste 3 Attribute
                                shown = 0
                                for key, value in attrs.items():
                                    print(f"         - {key}: {value}")
                    
                elif isinstance(creds_for_pres, dict):
                    # Dictionary mit referent als key
                    for referent, creds in creds_for_pres.items():
                        if isinstance(creds, list):
                            total_creds += len(creds)
                            print(f"   {referent}: {len(creds)} Credential(s)")
                            
                            # Zeige Details des ersten Credentials
                            if len(creds) > 0:
                                cred = creds[0]
                                if "cred_info" in cred:
                                    cred_info = cred["cred_info"]
                                    print(f"      • Cred Def ID: {cred_info.get('cred_def_id', 'N/A')}")
                                    attrs = cred_info.get("attrs", {})
                                    if attrs:
                                        attr_count = len(attrs)
                                        print(f"      • {attr_count} Attribute(s) verfügbar")
                else:
                    print(f"   ⚠️  Unbekanntes Format: {type(creds_for_pres).__name__}")
                    total_creds = 0
                
                print(f"\n✅ Total: {total_creds} passende Credential(s) gefunden")
                
                if total_creds == 0:
                    print(f"\n⚠️  WARNUNG: Keine passenden Credentials gefunden!")
                    print(f"   • Stelle sicher dass Cell 16 erfolgreich war")
                    print(f"   • Prüfe ob Credential im Holder Wallet ist (Cell 16 Output)")
                    print(f"   • Schema/Cred Def müssen mit Proof Request übereinstimmen")
            else:
                print(f"\n⚠️  Keine Credentials für Presentation verfügbar")
                print(f"   API-Antwort war leer oder None")
        else:
            print("⚠️  Kein Proof Request im Status 'request-received'")
            print("\n   Verfügbare Presentation Exchanges:")
            for pres in holder_pres_ex["results"]:
                print(f"      • {pres['pres_ex_id']} State: {pres.get('state')}")
    else:
        print("⚠️  Keine Presentation Exchanges gefunden")
        print("   Führe Cell 18 aus um Proof Request zu senden")
else:
    print("⚠️  Übersprungen: Kein Presentation Exchange vorhanden")
    print("   Führe Cell 18 aus um Proof Request zu erstellen")


📥 Holder: Proof Request verarbeiten...

✅ Holder hat Proof Request empfangen
   Presentation Exchange ID: ff0fac37-b79f-48e7-8546-c17149a3435a

📋 Verfügbare Credentials für Presentation:
   Gefunden: 1 Credential(s)

   Credential #1:
      • Referent: 673556ae-53fd-47a6-aaa9-f22d087814ca
      • Schema ID: 9pbXiFBZZGwXKp61HQBz3J:2:kritis_emergency_maintenance_cert:1.0
      • Cred Def ID: 9pbXiFBZZGwXKp61HQBz3J:3:CL:10:default
      • Attributes:
         - epoch_valid_from: 1765026000
         - security_access_level: Stufe-3-Kritisch
         - role: Notfalltechniker
         - first_name: Max
         - cert_type: Notfall-Wartungsberechtigung
         - epoch_valid_until: 1765033200
         - organisation: Musterfirma GmbH
         - facility_type: Umspannwerk Nord-Ost
         - name: Mustermann

✅ Total: 1 passende Credential(s) gefunden


In [246]:
# Cell 22: Holder - Presentation senden

print("📤 Holder: Presentation senden...\n")

if pres_ex_id and holder_pres_ex_id:
    start_time = time.time()
    
    # Hole Credentials nochmal
    creds_for_pres = api_get(
        HOLDER_ADMIN_URL,
        f"/present-proof-2.0/records/{holder_pres_ex_id}/credentials"
    )
    
    if creds_for_pres:
        print(f"📋 Credentials für Presentation gefunden")
        
        # Baue Presentation Request
        requested_credentials = {
            "indy": {
                "requested_attributes": {},
                "requested_predicates": {},
                "self_attested_attributes": {}
            }
        }
        
        # Handle verschiedene API-Response-Formate
        if isinstance(creds_for_pres, list):
            # Liste von Credential-Objekten
            print(f"   Format: Liste mit {len(creds_for_pres)} Credential(s)\n")
            
            # Jedes Credential-Objekt hat presentation_referents
            for cred_item in creds_for_pres:
                if "cred_info" in cred_item:
                    cred_id = cred_item["cred_info"].get("referent")
                    
                    # Hole presentation_referents (welche Felder dieses Cred erfüllen kann)
                    pres_referents = cred_item.get("presentation_referents", [])
                    
                    for ref in pres_referents:
                        # Prüfe ob es Attribute oder Predicate ist
                        if "attr" in ref or "attribute" in ref.lower():
                            requested_credentials["indy"]["requested_attributes"][ref] = {
                                "cred_id": cred_id,
                                "revealed": True
                            }
                            print(f"   ✓ Mapped attribute '{ref}' → Credential {cred_id}")
                        elif "pred" in ref or "predicate" in ref.lower():
                            requested_credentials["indy"]["requested_predicates"][ref] = {
                                "cred_id": cred_id
                            }
                            print(f"   ✓ Mapped predicate '{ref}' → Credential {cred_id}")
        
        elif isinstance(creds_for_pres, dict):
            # Dictionary mit referent als key (Legacy-Format)
            print(f"   Format: Dictionary mit {len(creds_for_pres)} Referent(s)\n")
            
            for referent, creds in creds_for_pres.items():
                if isinstance(creds, list) and len(creds) > 0:
                    first_cred = creds[0]
                    cred_id = first_cred.get("cred_info", {}).get("referent")
                    
                    if "attr" in referent:
                        requested_credentials["indy"]["requested_attributes"][referent] = {
                            "cred_id": cred_id,
                            "revealed": True
                        }
                        print(f"   ✓ Mapped attribute '{referent}' → Credential {cred_id[:20]}...")
                    elif "predicate" in referent:
                        requested_credentials["indy"]["requested_predicates"][referent] = {
                            "cred_id": cred_id
                        }
                        print(f"   ✓ Mapped predicate '{referent}' → Credential {cred_id[:20]}...")
        
        # Prüfe ob wir Credentials gemappt haben
        total_mapped = (
            len(requested_credentials["indy"]["requested_attributes"]) +
            len(requested_credentials["indy"]["requested_predicates"])
        )
        
        if total_mapped == 0:
            print("\n⚠️  WARNUNG: Keine Credentials gemappt!")
            print("   Versuche automatisches Mapping...\n")
            
            # Fallback: Verwende erstes verfügbares Credential für alle Requests
            # Hole Proof Request Details
            pres_ex_details = api_get(
                HOLDER_ADMIN_URL,
                f"/present-proof-2.0/records/{holder_pres_ex_id}"
            )
            
            if pres_ex_details and "pres_request" in pres_ex_details:
                pres_request = pres_ex_details["pres_request"]
                
                # Hole erstes verfügbares Credential
                first_cred_id = None
                if isinstance(creds_for_pres, list) and len(creds_for_pres) > 0:
                    first_cred_id = creds_for_pres[0].get("cred_info", {}).get("referent")
                elif isinstance(creds_for_pres, dict):
                    for creds in creds_for_pres.values():
                        if isinstance(creds, list) and len(creds) > 0:
                            first_cred_id = creds[0].get("cred_info", {}).get("referent")
                            break
                
                if first_cred_id:
                    # Map alle requested attributes
                    if "requested_attributes" in pres_request:
                        for attr_ref in pres_request["requested_attributes"].keys():
                            requested_credentials["indy"]["requested_attributes"][attr_ref] = {
                                "cred_id": first_cred_id,
                                "revealed": True
                            }
                            print(f"   ✓ Auto-mapped attribute '{attr_ref}' → {first_cred_id[:20]}...")
                    
                    # Map alle requested predicates
                    if "requested_predicates" in pres_request:
                        for pred_ref in pres_request["requested_predicates"].keys():
                            requested_credentials["indy"]["requested_predicates"][pred_ref] = {
                                "cred_id": first_cred_id
                            }
                            print(f"   ✓ Auto-mapped predicate '{pred_ref}' → {first_cred_id[:20]}...")
                    
                    total_mapped = (
                        len(requested_credentials["indy"]["requested_attributes"]) +
                        len(requested_credentials["indy"]["requested_predicates"])
                    )
        
        print(f"\n📊 Presentation Mapping:")
        print(f"   • Requested Attributes: {len(requested_credentials['indy']['requested_attributes'])}")
        print(f"   • Requested Predicates: {len(requested_credentials['indy']['requested_predicates'])}")
        print(f"   • Total Mapped: {total_mapped}")
        
        if total_mapped > 0:
            # Sende Presentation
            print("\n📤 Sende Presentation...")
            presentation_response = api_post(
                HOLDER_ADMIN_URL,
                f"/present-proof-2.0/records/{holder_pres_ex_id}/send-presentation",
                requested_credentials
            )
            
            if presentation_response is not None:
                print(f"\n✅ Holder hat Presentation gesendet")
                print(f"   Presentation Exchange ID: {holder_pres_ex_id}")
                print(f"   State: {presentation_response.get('state', 'N/A')}")
                print(f"   Role: {presentation_response.get('role', 'N/A')}")
                
                # Warte auf Verification
                print("\n   Warte auf Verification durch Verifier...")
                time.sleep(3)
                
                # Prüfe finalen Status (kann 404 geben wenn abgeschlossen!)
                final_status = api_get(
                    HOLDER_ADMIN_URL,
                    f"/present-proof-2.0/records/{holder_pres_ex_id}"
                )
                
                if final_status:
                    # Record noch vorhanden (--preserve-exchange-records aktiv)
                    final_state = final_status.get("state")
                    print(f"\n   📋 Finaler Holder Status: {final_state}")
                    
                    if final_state in ["done", "presentation-sent"]:
                        print(f"   ✅ Presentation erfolgreich abgeschlossen!")
                        print(f"   ✅ Record bleibt verfügbar (--preserve-exchange-records aktiv)")
                    elif final_state == "abandoned":
                        print(f"   ⚠️  Presentation wurde abgebrochen")
                    else:
                        print(f"   ⏳ Presentation in Bearbeitung...")
                else:
                    # 404 Error = Record wurde gelöscht = erfolgreich abgeschlossen!
                    print(f"\n   📋 Finaler Holder Status: completed/archived (404)")
                    print(f"   ✅ Presentation erfolgreich abgeschlossen!")
                    print(f"   ℹ️  Record wurde nach Completion gelöscht (--preserve-exchange-records nicht aktiv)")
                    print(f"\n   💡 Hinweis: Für dauerhafte Records, starte Holder neu:")
                    print(f"      docker-compose restart holder")
                
                duration = time.time() - start_time
                
                # Initialize key if not exists
                if "presentation" not in performance_metrics:
                    performance_metrics["presentation"] = []
                
                performance_metrics["presentation"].append(duration)
                print(f"\n⏱️  Presentation Zeit: {duration:.3f}s")
            else:
                print("\n❌ Fehler beim Senden der Presentation")
                print("   API-Response war leer oder fehlerhaft")
        else:
            print("\n❌ Kann Presentation nicht senden - keine Credentials gemappt")
            print("   • Prüfe ob Credentials die Proof Request Anforderungen erfüllen")
            print("   • Prüfe Schema und Cred Def IDs")
    else:
        print("⚠️  Keine Credentials für Presentation gefunden")
        print("   • Führe Cell 16 aus um Credential auszustellen")
        print("   • Prüfe ob Credential im Holder Wallet ist")
else:
    print("⚠️  Übersprungen: Presentation Exchange nicht vorhanden")
    if not pres_ex_id:
        print("   • Verifier Presentation Exchange ID fehlt (führe Cell 18 aus)")
    if not holder_pres_ex_id:
        print("   • Holder Presentation Exchange ID fehlt (führe Cell 19 aus)")


📤 Holder: Presentation senden...

📋 Credentials für Presentation gefunden
   Format: Liste mit 1 Credential(s)

   ✓ Mapped attribute 'attr1_referent' → Credential 673556ae-53fd-47a6-aaa9-f22d087814ca
   ✓ Mapped attribute 'attr6_referent' → Credential 673556ae-53fd-47a6-aaa9-f22d087814ca
   ✓ Mapped attribute 'attr4_referent' → Credential 673556ae-53fd-47a6-aaa9-f22d087814ca
   ✓ Mapped attribute 'attr2_referent' → Credential 673556ae-53fd-47a6-aaa9-f22d087814ca
   ✓ Mapped attribute 'attr3_referent' → Credential 673556ae-53fd-47a6-aaa9-f22d087814ca
   ✓ Mapped attribute 'attr5_referent' → Credential 673556ae-53fd-47a6-aaa9-f22d087814ca

📊 Presentation Mapping:
   • Requested Attributes: 6
   • Requested Predicates: 0
   • Total Mapped: 6

📤 Sende Presentation...

✅ Holder hat Presentation gesendet
   Presentation Exchange ID: ff0fac37-b79f-48e7-8546-c17149a3435a
   State: presentation-sent
   Role: prover

   Warte auf Verification durch Verifier...

   📋 Finaler Holder Status: done


In [250]:
# Cell 23: Verifier - Presentation verifizieren (mit Revocation Detection + Zeitgültigkeit)

print("✅ Presentation verifizieren (Verifier)...\n")

start_time = time.time()

# Wait for presentation to be received
import time as t
t.sleep(5)  # Give Holder time to generate and send proof

# Get presentation exchange record (ACA-Py endpoint)
# API: GET /present-proof-2.0/records/{pres_ex_id}
pres_ex_record = api_get(
    VERIFIER_ADMIN_URL,
    f"/present-proof-2.0/records/{pres_ex_id}"
)

if pres_ex_record is not None:
    # Response Format: {"state": "...", "verified": "...", "pres": {...}, "by_format": {...}, ...}
    pres_state = pres_ex_record.get("state")
    pres_verified = pres_ex_record.get("verified")

    # ========================================
    # ATTRIBUTE MAPPING: referent -> name -> value
    # ========================================
    # Step 1: Get requested_attributes from by_format.pres_request.indy (referent -> name mapping)
    by_format = pres_ex_record.get("by_format", {})
    pres_request_indy = by_format.get("pres_request", {}).get("indy", {})
    requested_attributes = pres_request_indy.get("requested_attributes", {})

    # Step 2: Get revealed_attrs from by_format.pres.indy.requested_proof (referent -> value mapping)
    pres_indy = by_format.get("pres", {}).get("indy", {})
    requested_proof = pres_indy.get("requested_proof", {})
    revealed_attrs_by_referent = requested_proof.get("revealed_attrs", {})

    # Step 3: Build name -> value mapping
    revealed_attrs = {}
    for referent, attr_data in revealed_attrs_by_referent.items():
        # Get the attribute name from requested_attributes
        if referent in requested_attributes:
            attr_name = requested_attributes[referent].get("name")
            attr_value = attr_data.get("raw")
            if attr_name and attr_value:
                revealed_attrs[attr_name] = attr_value

    duration = time.time() - start_time
    performance_metrics["proof_verification"].append(duration)

    print(f"✅ Proof erfolgreich verifiziert!")

    print(f"\nVerifizierte Attribute (REVEALED):")
    for name, value in revealed_attrs.items():
        print(f"   - {name}: {value}")

    print(f"\n⚠️  PER ZKP VERIFIZIERT ABER DURCH DATENSCHUTZ GESCHÜTZT (UNREVEALED):")
    print(f"   - Vorname: NICHT offengelegt (Zero-Knowledge-Proof)")
    print(f"   - Nachname: NICHT offengelegt (Zero-Knowledge-Proof)")
    print(f"   - Organisation: NICHT offengelegt (Zero-Knowledge-Proof)")

    print(f"\n   State:     {pres_state}")
    print(f"   Verified:  {pres_verified}")

    # ========================================
    # REVOCATION CHECK
    # ========================================
    is_revoked = False
    if pres_state == "done" and pres_verified == "true":
        print(f"\n✅ Credential ist NICHT revoked (gültig)")
    else:
        print(f"\n❌ Credential ist REVOKED!")
        is_revoked = True

    # ========================================
    # ZEITGÜLTIGKEITS-PRÜFUNG
    # ========================================
    print("\n" + "="*60)
    print("🕐 ZEITGÜLTIGKEITS-PRÜFUNG")
    print("="*60)

    current_epoch = 1765029600  # Beispiel: Fester Zeitpunkt für Test (2024-09-05 12:00:00 UTC)
    epoch_valid_from = None
    epoch_valid_until = None
    is_time_valid = False

    # Extrahiere epoch_valid_from und epoch_valid_until aus revealed_attrs (name -> value mapping)
    if "epoch_valid_from" in revealed_attrs:
        epoch_valid_from = int(revealed_attrs["epoch_valid_from"])

    if "epoch_valid_until" in revealed_attrs:
        epoch_valid_until = int(revealed_attrs["epoch_valid_until"])

    if epoch_valid_from is not None and epoch_valid_until is not None:
        # Konvertiere zu lesbaren Timestamps
        from datetime import datetime
        valid_from_dt = datetime.fromtimestamp(epoch_valid_from)
        valid_until_dt = datetime.fromtimestamp(epoch_valid_until)
        current_dt = datetime.fromtimestamp(current_epoch)

        print(f"   • Aktueller Zeitpunkt: {current_dt.strftime('%Y-%m-%d %H:%M:%S')} (Epoch: {current_epoch}) ==> Beispielwert")
        print(f"   • Gültig ab:            {valid_from_dt.strftime('%Y-%m-%d %H:%M:%S')} (Epoch: {epoch_valid_from})")
        print(f"   • Gültig bis:           {valid_until_dt.strftime('%Y-%m-%d %H:%M:%S')} (Epoch: {epoch_valid_until})")

        # Prüfe Zeitgültigkeit
        if epoch_valid_from <= current_epoch <= epoch_valid_until:
            is_time_valid = True
            print(f"\n   ✅ Zertifikat ist ZEITLICH GÜLTIG")
        else:
            is_time_valid = False
            if current_epoch < epoch_valid_from:
                print(f"\n   ❌ Zertifikat ist NOCH NICHT gültig (zu früh)")
            else:
                print(f"\n   ❌ Zertifikat ist ABGELAUFEN (zu spät)")
    else:
        print(f"   ⚠️  Zeitgültigkeits-Attribute nicht gefunden!")
        print(f"      • epoch_valid_from: {'gefunden' if epoch_valid_from else 'FEHLT'}")
        print(f"      • epoch_valid_until: {'gefunden' if epoch_valid_until else 'FEHLT'}")

    print("="*60)

    # ========================================
    # FINALE ZUGRIFFSENTSCHEIDUNG
    # ========================================
    print("\n" + "="*60)
    print("🚦 FINALE ZUGRIFFSENTSCHEIDUNG")
    print("="*60)

    # Beide Bedingungen müssen erfüllt sein
    if not is_revoked and is_time_valid:
        print(f"\n✅✅ ZUGANG GEWÄHRT")
        print(f"   ✓ Credential ist gültig (nicht revoked)")
        print(f"   ✓ Zertifikat ist zeitlich gültig")
        print(f"\n   🔓 Zugang zum Umspannwerk Nord-Ost GEWÄHRT")
    else:
        print(f"\n❌❌ ZUGANG VERWEIGERT")
        if is_revoked:
            print(f"   ✗ Credential ist REVOKED")
        if not is_time_valid:
            print(f"   ✗ Zertifikat ist NICHT zeitlich gültig")
        print(f"\n   🔒 Zugang zum Umspannwerk Nord-Ost VERWEIGERT")

    print("="*60)

    print(f"\n✅ Privacy-Preserving Verification erfolgreich (DSGVO-konform)")
    print(f"⏱️  Zeit: {duration:.3f}s")

    # Show full Presentation Record
    pretty_print(pres_ex_record, "Presentation Record (KRITIS)")

else:
    print("❌ Fehler beim Abrufen der Presentation")

✅ Presentation verifizieren (Verifier)...

✅ Proof erfolgreich verifiziert!

Verifizierte Attribute (REVEALED):
   - role: Notfalltechniker
   - epoch_valid_from: 1765026000
   - cert_type: Notfall-Wartungsberechtigung
   - facility_type: Umspannwerk Nord-Ost
   - epoch_valid_until: 1765033200
   - security_access_level: Stufe-3-Kritisch

⚠️  PER ZKP VERIFIZIERT ABER DURCH DATENSCHUTZ GESCHÜTZT (UNREVEALED):
   - Vorname: NICHT offengelegt (Zero-Knowledge-Proof)
   - Nachname: NICHT offengelegt (Zero-Knowledge-Proof)
   - Organisation: NICHT offengelegt (Zero-Knowledge-Proof)

   State:     done
   Verified:  true

✅ Credential ist NICHT revoked (gültig)

🕐 ZEITGÜLTIGKEITS-PRÜFUNG
   • Aktueller Zeitpunkt: 2025-12-06 14:00:00 (Epoch: 1765029600) ==> Beispielwert
   • Gültig ab:            2025-12-06 13:00:00 (Epoch: 1765026000)
   • Gültig bis:           2025-12-06 15:00:00 (Epoch: 1765033200)

   ✅ Zertifikat ist ZEITLICH GÜLTIG

🚦 FINALE ZUGRIFFSENTSCHEIDUNG

✅✅ ZUGANG GEWÄHRT
   ✓ C

## Teil 7: Revocation - Beendigung der Wartungsberechtigung

**Szenario**: Der Notfall-Wartungseinsatz ist beendet. Der Energienetzbetreiber (Issuer) widerruft das Credential des Technikers, um dessen Zugang zur kritischen Infrastruktur zu beenden.

**KRITIS-Sicherheitsaspekt**:
- **Just-in-Time Access**: Berechtigungen nur für die Dauer des Einsatzes
- **Sofortige Revocation**: Credential wird in Revocation Registry als "revoked" markiert
- **Ledger-Publikation**: Revocation wird auf Hyperledger Indy Blockchain veröffentlicht
- **Echtzeit-Validierung**: Zutrittssysteme prüfen Revocation-Status bei jeder Authentifizierung

**Demonstration**:
1. Credential wird vom Betreiber revoked (Cell 25-26)
2. Techniker versucht erneut Zugang (Cell 27)
3. Zutrittssystem VERWEIGERT Zugang (Revocation erkannt)

In [ ]:
# Cell 24: Revocation Registries anzeigen

print("="*60)
print("📋 ISSUER: Revocation Registries")
print("="*60 + "\n")

registries = api_get(
    ISSUER_ADMIN_URL,
    "/revocation/registries/created?state=active"
)

if registries and "rev_reg_ids" in registries:
    reg_ids = registries["rev_reg_ids"]
    print(f"✅ Aktive Revocation Registries: {len(reg_ids)}\n")
    
    for idx, reg_id in enumerate(reg_ids, 1):
        print(f"Registry #{idx}: {reg_id}")
        
        # Registry Details abrufen
        reg_info = api_get(ISSUER_ADMIN_URL, f"/revocation/registry/{reg_id}").get('result', 'N/A')
        if reg_info:
            print(f"   • State: {reg_info.get('state')}")
            print(f"   • Max Credentials: {reg_info.get('max_cred_num')}")
            print(f"   • Issuer DID: {reg_info.get('issuer_did', 'N/A')}")
            print(f"   • Tails Hash: {reg_info.get('tails_hash', 'N/A')}")
            print(f"   • Tails Location: {reg_info.get('tails_local_path', 'N/A')}")
            print(f"   • Created: {reg_info.get('created_at', 'N/A')}\n")
else:
    print("⚠️  Keine aktiven Revocation Registries gefunden")
    print("   • Stelle sicher dass Cell 12 mit support_revocation: True läuft")
    print("   • Führe Cell 16 aus um Credential mit Revocation auszustellen")

print("="*60)


In [ ]:
# Cell 25: Credential REVOKEN (staged)

print("="*60)
print("🚫 Credential REVOKEN (staged)")
print("="*60 + "\n")

# Verwende gespeicherte Werte aus Cell 16
if 'rev_reg_id' in locals() and 'cred_rev_id' in locals():
    print(f"📋 Zu revokendes Credential:")
    print(f"   • Rev Reg ID: {rev_reg_id}")
    print(f"   • Cred Rev ID: {cred_rev_id}\n")
    
    revoke_request = {
        "rev_reg_id": rev_reg_id,
        "cred_rev_id": cred_rev_id,
        "notify": False,  # Kein Notification-Webhook
        "publish": False,  # Staging only, publishe später in Cell 31
        "comment": "Revoked for testing purposes"
    }
    
    print("📤 Sende Revocation Request (staging)...")
    start_time = time.time()
    
    revoke_response = api_post(
        ISSUER_ADMIN_URL,
        "/revocation/revoke",
        revoke_request
    )
    
    duration = time.time() - start_time
    
    if revoke_response is not None:
        print(f"\n✅ Credential erfolgreich REVOKED (staged)")
        print(f"   • Status: Pending (noch nicht auf Ledger)")
        print(f"   • Zeit: {duration:.3f}s")
        print(f"\n💡 Nächster Schritt: Führe Cell 31 aus um auf Ledger zu publishen!")
    else:
        print(f"\n❌ Revocation fehlgeschlagen")
        print(f"   Prüfe ob die IDs korrekt sind")
else:
    print("⚠️  Keine Revocation IDs gefunden!")
    print("   • Führe Cell 16 aus um Credential auszustellen")
    print("   • Die IDs werden automatisch in rev_reg_id und cred_rev_id gespeichert")

print("\n" + "="*60)


In [ ]:
# Cell 26: Revocations auf Ledger PUBLISHEN

print("="*60)
print("📤 PUBLISH Revocations auf Ledger")
print("="*60 + "\n")

print("Publishing alle pending Revocations auf Ledger...")
start_time = time.time()

publish_response = api_post(
    ISSUER_ADMIN_URL,
    "/revocation/publish-revocations",
    {}
)

duration = time.time() - start_time

if publish_response and "rrid2crid" in publish_response:
    revoked_regs = publish_response["rrid2crid"]
    total_revoked = sum(len(creds) for creds in revoked_regs.values())
    
    print(f"\n✅ ERFOLGREICH auf Ledger published:")
    print(f"   • Revocation Registries: {len(revoked_regs)}")
    print(f"   • Total revoked Credentials: {total_revoked}")
    print(f"   • Publish Zeit: {duration:.3f}s\n")
    
    for reg_id, cred_ids in revoked_regs.items():
        print(f"   📋 Registry: {reg_id}")
        print(f"      Revoked Credential IDs: {cred_ids}")
        print()
    
    print("="*60)
    print("✅ Revocations sind jetzt auf dem VON Ledger!")
    print("   💡 Holder kann nun KEINEN Non-Revocation-Proof mehr erstellen")
    print("="*60)
elif publish_response:
    print(f"\n⚠️  Keine pending Revocations zum Publishen")
    print(f"   Response: {publish_response}")
else:
    print(f"\n❌ Publish fehlgeschlagen")

print("Jetzt erneut Cell 19 - 23 ausführen um Revocation zu testen!")


In [ ]:
# Cell 27: Proof Request MIT Revocation-Check

print("🔍 Proof Request MIT Revocation-Check senden...\n")

start_time = time.time()

# Get current timestamp for non-revocation interval
current_timestamp = int(time.time())

# Proof Request Data (ACA-Py API) - WITH REVOCATION CHECK
# API: POST /present-proof-2.0/send-request
proof_request_revoked_data = {
    "connection_id": conn_id_verifier_holder,
    "presentation_request": {
        "indy": {
            "name": "Proof of KRITIS Access Authorization (Non-Revoked Check)",
            "version": "1.0",
            "requested_attributes": {
                "attr1_referent": {
                    "name": "first_name",
                    "restrictions": [{"cred_def_id": cred_def_id}],
                    "non_revoked": {"from": 0, "to": current_timestamp}
                },
                "attr2_referent": {
                    "name": "name",
                    "restrictions": [{"cred_def_id": cred_def_id}],
                    "non_revoked": {"from": 0, "to": current_timestamp}
                },
                "attr3_referent": {
                    "name": "cert_type",
                    "restrictions": [{"cred_def_id": cred_def_id}],
                    "non_revoked": {"from": 0, "to": current_timestamp}
                },
                "attr4_referent": {
                    "name": "facility_type",
                    "restrictions": [{"cred_def_id": cred_def_id}],
                    "non_revoked": {"from": 0, "to": current_timestamp}
                }
            },
            "requested_predicates": {}
        }
    }
}

print(f"🔍 Proof Request Parameter:")
print(f"   Proof Name:    Proof of KRITIS Access Authorization (Non-Revoked Check)")
print(f"   Non-Revoked:   {{from: 0, to: {current_timestamp}}}")
print(f"   ⚠️  Erwartetes Ergebnis: Zutrittssystem VERWEIGERT Zugang (Credential revoked)\n")

# Send Proof Request
proof_request_revoked_response = api_post(
    VERIFIER_ADMIN_URL,
    "/present-proof-2.0/send-request",
    proof_request_revoked_data
)

if proof_request_revoked_response is not None:
    pres_ex_id_revoked = proof_request_revoked_response.get("pres_ex_id")
    pres_ex_state_revoked = proof_request_revoked_response.get("state")

    duration = time.time() - start_time
    performance_metrics["proof_request"].append(duration)

    print(f"✅ Proof Request NACH Revocation gesendet")
    print(f"   Exchange ID: {pres_ex_id_revoked}")
    print(f"   State:       {pres_ex_state_revoked}")
    print(f"⏱️  Zeit: {duration:.3f}s")

    # Show full Proof Request Response
    pretty_print(proof_request_revoked_response, "Proof Request Response (nach Revocation)")

else:
    print("❌ Fehler beim Senden des Proof Requests")

In [ ]:
# Cell 28: Presentation Verification (mit Revocation-Check)

print("🚫 Presentation verifizieren (Erwarteter Fehler wegen Revocation)...\n")

start_time = time.time()

# Wait for presentation attempt
import time as t
t.sleep(10)  # Give Holder time to attempt proof generation

# Get presentation exchange record
pres_ex_record_revoked = api_get(
    VERIFIER_ADMIN_URL,
    f"/present-proof-2.0/records/{pres_ex_id_revoked}"
)

if pres_ex_record_revoked is not None:
    pres_state_revoked = pres_ex_record_revoked.get("state")
    pres_verified_revoked = pres_ex_record_revoked.get("verified")

    duration = time.time() - start_time
    performance_metrics["proof_verification"].append(duration)

    if pres_verified_revoked == "true" or pres_state_revoked == "done":
        print("⚠️  WARNUNG: Proof wurde verifiziert (SOLLTE NICHT PASSIEREN!)")
    else:
        print("🚫 ZUGANG ZUM UMSPANNWERK VERWEIGERT\n")
        print("   Grund: Credential ist revoked\n")
        print("   Details:")
        print("   - Credential wurde vom Betreiber widerrufen")
        print("   - Revocation ist auf Ledger publiziert")
        print("   - Non-Revocation-Proof kann nicht erstellt werden")
        print("   - Techniker darf Anlage NICHT mehr betreten\n")
        print("   ✅ KRITIS-Sicherheitsmechanismus funktioniert korrekt!")
        print("   ✅ Just-in-Time Access erfolgreich demonstriert")

    print(f"\n   State:     {pres_state_revoked}")
    print(f"   Verified:  {pres_verified_revoked}")
    print(f"⏱️  Zeit: {duration:.3f}s")

    # Show full Presentation Record
    pretty_print(pres_ex_record_revoked, "Presentation Record (nach Revocation)")

else:
    print("❌ Fehler beim Abrufen der Presentation")

In [ ]:
# Cell 29: Revocation Registry ROTATION (Optional)

print("="*60)
print("🔄 Revocation Registry ROTATION")
print("="*60 + "\n")

print("💡 Registry Rotation ist nötig wenn:")
print("   • Registry voll ist (100 Credentials mit TAILS_FILE_COUNT=100)")
print("   • Neue Registry für weitere Credentials benötigt wird\n")

if 'cred_def_id' in locals() and cred_def_id:
    print(f"Rotiere Registry für Cred Def: {cred_def_id[:50]}...\n")
    
    start_time = time.time()
    
    rotate_response = api_post(
        ISSUER_ADMIN_URL,
        f"/revocation/active-registry/{cred_def_id}/rotate",
        {}
    )
    
    duration = time.time() - start_time
    
    if rotate_response and "rev_reg_ids" in rotate_response:
        decom = rotate_response["rev_reg_ids"]
        print(f"✅ Registry erfolgreich rotiert!")
        print(f"   • Decommissioned Registries: {len(decom)}")
        for reg_id in decom:
            print(f"      - {reg_id[:50]}...")
        print(f"   • Zeit: {duration:.3f}s")
        print(f"\n   💡 Neue aktive Registry wurde erstellt")
        print(f"   💡 Weitere Credentials können jetzt ausgestellt werden")
    else:
        print(f"⚠️  Rotation nicht möglich")
        print(f"   • Möglicherweise ist die Registry noch nicht voll")
        print(f"   • Oder es gibt keine aktive Registry für diese Cred Def")
        if rotate_response:
            print(f"   Response: {rotate_response}")
else:
    print("⚠️  Keine Credential Definition ID gefunden")
    print("   Führe Cell 12 aus um Cred Def zu erstellen")

print("\n" + "="*60)
